# Modelling

In [131]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tqdm.keras import TqdmCallback
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import pickle
import unicodedata
from sklearn.metrics import classification_report

# ===============================================
# CONFIGURASI UTAMA
# ===============================================
# PASTIKAN FILE PATH SESUAI DENGAN NAMA FILE ANDA
FILE_PATH = "final_production_judol_detection.csv" 
MAX_WORDS = 20000
MAX_LEN = 100
EMBED_DIM = 128
BATCH_SIZE = 64
EPOCHS = 25
LR = 1e-3
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
TEST_SIZE_FINAL = 0.20
NEW_ALPHA = 0.6 # Alpha untuk Focal Loss, memberi bobot lebih ke kelas Judol (1)
OPTIMAL_THRESHOLD = 0.65 
TEXT_COLUMN = 'cleaned_comment_text'
TARGET_COLUMN = 'target'

# ===============================================
# 1. DATA AUGMENTATION: SINTESIS JUDOL BARU
# ===============================================

# Pola Brand & Kata Kunci
prefixes = ['slot', 'jp', 'wd', 'depo', 'gacor', 'raja', 'bosku', 'mantap']
numbers = [str(i) for i in range(100, 999, 100)] + [str(i) for i in range(11, 99, 11)]
phrases = ['gacor banget di sini!', 'pasti jp di situs ini', 'wd cepat tanpa ribet', 'cair terus bosku']

synthetic_data = []
for i in range(200): 
    prefix = np.random.choice(prefixes)
    num = np.random.choice(numbers)
    phrase = np.random.choice(phrases)
    
    brand_name = prefix + num
    comment = f"{phrase} coba main di {brand_name} aja deh"
    
    # Cleaning dasar untuk data sintetis
    cleaned_comment = re.sub(r'\s+', ' ', re.sub(r"[^a-z0-9\s]", " ", comment.lower())).strip()
    
    synthetic_data.append({
        "comment_text": comment,
        "target": 1,
        "cleaned_comment_text": cleaned_comment,
        "combined_text": cleaned_comment + " " + comment 
    })

df_synthetic = pd.DataFrame(synthetic_data)
df_synthetic['cleaned_comment_text'] = df_synthetic['cleaned_comment_text'].str.replace('coba main di ', '').str.replace(' aja deh', '')

# ===============================================
# 2. LOAD & GABUNGKAN DATA
# ===============================================

def merge_spaced_characters(text):
    """
    Menggabungkan pola huruf/angka yang dipisah spasi namun merupakan satu kata.
    Contoh: 'p u l a u w i n 8 8' -> 'pulauwin88'
    """
    tokens = text.split()
    merged_tokens = []
    buffer = []

    def flush_buffer():
        if len(buffer) >= 3:  # Minimal 3 karakter untuk dianggap kata terpisah
            merged_tokens.append("".join(buffer))
        else:
            merged_tokens.extend(buffer)
        buffer.clear()

    for tok in tokens:
        if len(tok) == 1 and re.match(r"[a-zA-Z0-9]", tok):
            buffer.append(tok.lower())
        else:
            flush_buffer()
            merged_tokens.append(tok)

    flush_buffer()
    return " ".join(merged_tokens)

def clean_text_prep(text):
    """Fungsi cleaning yang digunakan untuk Tokenisasi."""
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    
    text = merge_spaced_characters(text)
    
    # NFKD: Mengubah karakter fancy (misal 𝗛𝗢𝗧𝐉𝐄𝐏𝐄𝟱𝟱) menjadi bentuk dasarnya (HOTJEPE55)
    text = unicodedata.normalize('NFKD', text)
    
    # Encode/Decode: Menghapus semua karakter non-ASCII (seperti emoji 🀄🥰, diakritik)
    # yang tersisa setelah NFKD, hanya menyisakan karakter dasar ASCII.
    # Ini adalah langkah kunci untuk mengatasi masalah hilangnya karakter fancy.
    text = text.encode('ascii', 'ignore').decode('utf-8')
    
    # 3. Lowercase (Dapat dilakukan setelah konversi ASCII)
    text = text.lower()
    
    # 4. Hapus sisa karakter yang bukan a-z, 0-9, atau spasi
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    
    # 5. Rapikan spasi
    text = re.sub(r"\s+", " ", text).strip()
    return text

print("--- 1. Memuat Data, Augmentasi, dan Preprocessing ---")
try:
    df_original = pd.read_csv(FILE_PATH)
    # Pastikan kolom utama ada dan target numerik
    df_original = df_original.dropna(subset=[TEXT_COLUMN, TARGET_COLUMN])
    df_original[TARGET_COLUMN] = df_original[TARGET_COLUMN].astype(int)
    
    # Gabungkan data asli dengan data sintetik
    df_combined = pd.concat([df_original[['comment_text', 'cleaned_comment_text', 'target', 'combined_text']], df_synthetic], ignore_index=True)

    # Bersihkan Teks pada kolom fitur utama
    df_combined[TEXT_COLUMN] = df_combined[TEXT_COLUMN].fillna('').apply(clean_text_prep)
    
    # Filter data yang teksnya tidak kosong
    df_combined = df_combined[df_combined[TEXT_COLUMN] != '']

except FileNotFoundError:
    print(f"❌ ERROR: File '{FILE_PATH}' tidak ditemukan.")
    exit()

X = df_combined[TEXT_COLUMN]
Y = df_combined[TARGET_COLUMN]

print(f"Total data setelah augmentasi dan cleaning: {len(X)} baris.")
print(f"Jumlah Judol (1) sebelum split: {Y.sum()}")

# ===============================================
# 3. SPLIT DATA (STRATIFIED)
# ===============================================
print("\n--- 2. Split Data (80% Train, 20% Test Stratified) ---")
# Pembagian 80% Train, 20% Test Murni
X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    X, 
    Y, 
    test_size=TEST_SIZE_FINAL, 
    random_state=SEED, 
    stratify=Y 
)

# ===============================================
# 4. FEATURE EXTRACTION (Tokenisasi & Padding)
# ===============================================
print("\n--- 3. Feature Extraction: Tokenisasi dan Padding ---")
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text.astype(str))

# SIMPAN TOKENIZER
with open('tokenizer_augmented.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Ubah Data Latih menjadi Vektor Fitur
X_train_seq = tokenizer.texts_to_sequences(X_train_text.astype(str))
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# Ubah Data Uji (Murni) menjadi Vektor Fitur
X_test_seq = tokenizer.texts_to_sequences(X_test_text.astype(str))
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')


# ===============================================
# 5. TERAPKAN SMOTE (HANYA PADA DATA LATIH)
# ===============================================
print("\n--- 4. Terapkan SMOTE (Oversampling pada Data Latih) ---")
Y_train_np = Y_train.values 
smote = SMOTE(sampling_strategy='minority', random_state=SEED)

# SMOTE diterapkan pada fitur numerik (X_train_padded) dan labelnya (Y_train_np)
X_resampled, Y_resampled = smote.fit_resample(X_train_padded, Y_train_np)

# Hasil Akhir (Data Latih yang Sudah Seimbang)
X_train_final = np.array(X_resampled)
Y_train_final = np.array(Y_resampled)

# Verifikasi Rasio Kelas
judol_count = Y_train_final.sum()
non_judol_count = len(Y_train_final) - judol_count
print(f"✅ Data Latih Akhir Setelah SMOTE: {len(X_train_final)} baris (Rasio 1:{round(non_judol_count / judol_count)})")


# ===============================================
# 6. TRAINING DENGAN DATA SMOTE
# ===============================================
def focal_loss(gamma=2., alpha=NEW_ALPHA):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        loss = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
                 - (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss)
    return loss
    
model_aug = Sequential([
    Embedding(MAX_WORDS, EMBED_DIM, input_length=MAX_LEN),
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
    GlobalMaxPool1D(),
    Dense(128, activation="relu"),
    Dropout(0.4),
    Dense(1, activation="sigmoid")
])

model_aug.compile(loss=focal_loss(), optimizer=Adam(learning_rate=LR), metrics=["accuracy"])

callbacks = [
    EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
    TqdmCallback(verbose=1),
]

print(f"\nMemulai Pelatihan Model dengan Data SMOTE (Alpha={NEW_ALPHA})...")

history = model_aug.fit(
    X_train_final, Y_train_final, # <-- MENGGUNAKAN DATA HASIL SMOTE
    validation_split=0.1, # Menggunakan 10% dari data SMOTE untuk validasi
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    # class_weight TIDAK DIPERLUKAN KARENA DATA SUDAH SEIMBANG OLEH SMOTE
    callbacks=callbacks,
    verbose=0
)

# SAVE MODEL BARU
NEW_MODEL_AUG_PATH = "judol_detection_augmented_smote.h5"
model_aug.save(NEW_MODEL_AUG_PATH)
print(f"\nModel baru disimpan sebagai {NEW_MODEL_AUG_PATH} ✅")


# ===============================================
# 7. EVALUASI DAN UJI MODEL BARU (KODE PERBAIKAN)
# ===============================================

print("\n=============================================")
print("📊 EVALUASI AKHIR MODEL PADA DATA UJI MURNI")
print("=============================================")

# FIX: Konversi Y_test (Pandas Series) menjadi NumPy Array untuk menghindari ValueError
Y_test_np = Y_test.values 

# Evaluasi Model pada Data Uji Murni (X_test_padded)
loss, accuracy = model_aug.evaluate(X_test_padded, Y_test_np, verbose=0) 

print(f"Loss pada Data Uji: {loss:.4f}")
print(f"Akurasi pada Data Uji: {accuracy:.4f}")

# Hitung Precision, Recall, dan F1-Score menggunakan OPTIMAL_THRESHOLD
y_pred_proba = model_aug.predict(X_test_padded, verbose=0)
y_pred_classes = (y_pred_proba > OPTIMAL_THRESHOLD).astype(int)

print(f"\nLaporan Klasifikasi (Threshold={OPTIMAL_THRESHOLD}):")
print(classification_report(Y_test_np, y_pred_classes)) # Menggunakan Y_test_np


# ===============================================
# 8. UJI MANUAL PADA KASUS SULIT (Pure ML)
# ===============================================

def clean_text(text):
    """Fungsi cleaning yang digunakan saat prediksi (dengan normalisasi Unicode)"""
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # Normalisasi Unicode
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') 
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def predict_text_ml_only(text, model, tokenizer, threshold=OPTIMAL_THRESHOLD):
    """Melakukan prediksi Judol murni tanpa Heuristik."""
    
    text_clean = clean_text(text)
    
    seq = tokenizer.texts_to_sequences([text_clean])
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    
    pred_prob = model.predict(pad, verbose=0)[0][0]
    is_judol = pred_prob > threshold
    
    label = "🔥 **JUDOL (SMOTE ML)**" if is_judol else "✅ Bukan Judol (SMOTE ML)"
    
    print(f"---")
    print(f"💬 Input: '{text}'")
    print(f"🧹 Clean: '{text_clean}'")
    print(f"🎯 Prediksi: {label} (Probabilitas: {pred_prob:.3f} | T={threshold})")

manual_data = [
    "WD 15 juta di BOSKU777 langsung cair tanpa ribet!", 
    "buruan daftar di MANTAPJEPE888 promonya gede banget", 
    "gacor di situs ini, main di kaisar999 depo 50 ribu wd 500 ribu", 
    "main di mahadewa pasti cair",
    "Ini brand baru: GACORWD77. Pasti JP!", 
    "Coba aja main di brand baru SLOTBARU555, depo langsung wd", 
    "nonton film gratis di youtube channel ini aja",
]

print("\n=============================================")
print(f"🚀 UJI COBA MODEL MURNI ML (SMOTE Trained, Threshold={OPTIMAL_THRESHOLD})")
print("=============================================")
for data in manual_data:
    predict_text_ml_only(data, model_aug, tokenizer)

--- 1. Memuat Data, Augmentasi, dan Preprocessing ---
Total data setelah augmentasi dan cleaning: 19444 baris.
Jumlah Judol (1) sebelum split: 1633

--- 2. Split Data (80% Train, 20% Test Stratified) ---

--- 3. Feature Extraction: Tokenisasi dan Padding ---

--- 4. Terapkan SMOTE (Oversampling pada Data Latih) ---
✅ Data Latih Akhir Setelah SMOTE: 28498 baris (Rasio 1:1)


/home/wtf/anaconda3/lib/python3.13/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


0epoch [00:00, ?epoch/s]

0batch [00:00, ?batch/s]


Memulai Pelatihan Model dengan Data SMOTE (Alpha=0.6)...

Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.

Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.



Model baru disimpan sebagai judol_detection_augmented_smote.h5 ✅

📊 EVALUASI AKHIR MODEL PADA DATA UJI MURNI
Loss pada Data Uji: 0.5560
Akurasi pada Data Uji: 0.5739

Laporan Klasifikasi (Threshold=0.65):
              precision    recall  f1-score   support

           0       1.00      0.58      0.74      3562
           1       0.18      0.97      0.30       327

    accuracy                           0.62      3889
   macro avg       0.59      0.78      0.52      3889
weighted avg       0.93      0.62      0.70      3889


🚀 UJI COBA MODEL MURNI ML (SMOTE Trained, Threshold=0.65)
---
💬 Input: 'WD 15 juta di BOSKU777 langsung cair tanpa ribet!'
🧹 Clean: 'wd 15 juta di bosku777 langsung cair tanpa ribet'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.989 | T=0.65)
---
💬 Input: 'buruan daftar di MANTAPJEPE888 promonya gede banget'
🧹 Clean: 'buruan daftar di mantapjepe888 promonya gede banget'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.987 | T=0.65)
---
💬 Input: 'gacor d

In [133]:
# file: robust_judol_training_and_prediction.py
# Versi: Robust augmentation + unified preprocessing + training + evaluation + prediction util
# Tujuan: model lebih kebal terhadap pola obfuscation yang tidak ada di dataset.

import os
import re
import random
import unicodedata
import string
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GlobalMaxPool1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# =========================
# CONFIG
# =========================
FILE_PATH = "final_production_judol_detection.csv"
MAX_WORDS = 30000
MAX_LEN = 100
EMBED_DIM = 128
BATCH_SIZE = 64
EPOCHS = 15
LR = 1e-3
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
TEST_SIZE_FINAL = 0.20
NEW_ALPHA = 0.55
OPTIMAL_THRESHOLD = 0.65
TEXT_COLUMN = 'combined_text'
TARGET_COLUMN = 'target'

# Paths to save
TOKENIZER_PATH = "tokenizer_augmented_robust.pickle"
MODEL_PATH = "judol_detection_augmented_smote_robust.keras"  # prefer native Keras format

# =========================
# UTIL: zero-width remover, fancy-char normalizer, etc.
# =========================
ZERO_WIDTH_CHARS = [
    '\u200b', '\u200c', '\u200d', '\u2060', '\uFEFF'
]

def remove_zero_width(text: str) -> str:
    for z in ZERO_WIDTH_CHARS:
        text = text.replace(z, '')
    return text

def normalize_unicode(text: str) -> str:
    # Turn fancy fonts into decomposed forms, then drop non-ascii
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

# =========================
# STRONG MERGE: merge sequences of single-chars separated by spaces/dots/underscores/other separators
# - handles "p u l a u w i n 8 8", "p.u.l.a.u.w.i.n", "p_u_l_a_u", "p u l a u • w i n"
# - only merges sequences >= min_len (default 3) to avoid accidental merges
# =========================
SEP_CHARS = r"[\s\.\-_\|•·\u2063]"  # add common separators (space,dot, dash, underscore, pipe, bullet, invisible joiner)
MIN_MERGE_LEN = 3

def merge_obfuscated_sequences(text: str, min_len=MIN_MERGE_LEN) -> str:
    # Replace runs like: a . b . c . 1 2 -> abc12
    # Strategy: find sequences of single-letter/digit tokens separated by sep chars and merge them
    # We'll token-scan (character-level) but mindful of real words
    tokens = re.split(r'(\s+)', text)  # keep whitespace tokens to preserve boundaries
    out_tokens = []
    buffer = []
    for tok in tokens:
        # If tok is whitespace, flush and keep it
        if tok.isspace():
            if buffer:
                # decide to merge or keep
                if len(buffer) >= min_len and all(re.fullmatch(r'[A-Za-z0-9]', c) or re.fullmatch(SEP_CHARS, c) for c in "".join(buffer)):
                    # normalize: remove separators, join alnum
                    merged = re.sub(SEP_CHARS, '', "".join(buffer))
                    out_tokens.append(merged)
                else:
                    out_tokens.append("".join(buffer))
                buffer = []
            out_tokens.append(tok)
            continue

        # If token is composed of single letters/digits separated by sep inside (like 'p.u.l.a'), detect:
        inner_chars = list(tok)
        # Heuristic: if token length small-ish and contains separators or only single chars separated,
        # add to buffer as sequence chunk
        if re.fullmatch(r'(?:[A-Za-z0-9]|' + SEP_CHARS + r')+$', tok):
            # put token into buffer char by char
            buffer.append(tok)
        else:
            # flush buffer
            if buffer:
                if len(buffer) >= min_len:
                    merged = re.sub(SEP_CHARS, '', "".join(buffer))
                    out_tokens.append(merged)
                else:
                    out_tokens.append("".join(buffer))
                buffer = []
            out_tokens.append(tok)
    # final flush
    if buffer:
        if len(buffer) >= min_len:
            merged = re.sub(SEP_CHARS, '', "".join(buffer))
            out_tokens.append(merged)
        else:
            out_tokens.append("".join(buffer))

    return "".join(out_tokens)

# Alternative simpler merge for 'space-separated tokens' e.g. "p u l a u"
def merge_spaced_characters_simple(text: str, min_len=MIN_MERGE_LEN) -> str:
    tokens = text.split()
    merged = []
    buffer = []
    for t in tokens:
        if len(t) == 1 and re.match(r'[A-Za-z0-9]', t):
            buffer.append(t)
        else:
            if len(buffer) >= min_len:
                merged.append("".join(buffer))
            elif buffer:
                merged.extend(buffer)
            buffer = []
            merged.append(t)
    if buffer:
        if len(buffer) >= min_len:
            merged.append("".join(buffer))
        else:
            merged.extend(buffer)
    return " ".join(merged)

# We'll apply both merge strategies (simple spaced + sep-aware) in pipeline.

# =========================
# AUGMENTATION HELPERS: create obfuscated variants for a base brand/text
# - spaced, dotted, underscored, insert zero-width, leet sub, random splits
# =========================
LEET_MAP = str.maketrans({
    'a': '4', 'o': '0', 'i': '1', 'e': '3', 's': '5', 't': '7', 'b': '8', 'g': '9'
})

def to_leet(s: str):
    return s.translate(LEET_MAP)

def spaced_variant(s: str):
    return " ".join(list(s))

def dotted_variant(s: str):
    return ".".join(list(s))

def underscored_variant(s: str):
    return "_".join(list(s))

def zero_width_variant(s: str):
    # insert zero-width joiner at random positions
    zw = '\u200b'
    out = []
    for ch in s:
        out.append(ch)
        if random.random() < 0.25:
            out.append(zw)
    return "".join(out)

def random_split_variant(s: str):
    # randomly insert spaces inside the word (simulate weird splits)
    out = []
    for ch in s:
        out.append(ch)
        if random.random() < 0.18:
            out.append(" ")
    return "".join(out)

def punctuated_variant(s: str):
    # insert random separators from a set
    seps = ['.', '-', '_', '·', '•', '|']
    out = []
    for ch in s:
        out.append(ch)
        if random.random() < 0.18:
            out.append(random.choice(seps))
    return "".join(out)

def generate_obfuscations(s: str, n=6):
    variants = set()
    variants.add(s)
    variants.add(s.lower())
    variants.add(to_leet(s.lower()))
    variants.add(spaced_variant(s.lower()))
    variants.add(dotted_variant(s.lower()))
    variants.add(underscored_variant(s.lower()))
    variants.add(zero_width_variant(s.lower()))
    variants.add(random_split_variant(s.lower()))
    variants.add(punctuated_variant(s.lower()))
    # trim and unique
    variants = list(v for v in variants if v and len(v) > 0)
    random.shuffle(variants)
    return variants[:n]

# =========================
# CLEANING PIPELINE (UNIFIED FOR TRAIN & PREDICT)
# Steps:
# 1) remove URLs
# 2) remove zero-width
# 3) normalize unicode -> ascii
# 4) apply merge_spaced_characters_simple (space-only)
# 5) apply merge_obfuscated_sequences (sep-aware)
# 6) lowercase, remove remaining non-alnum
# 7) collapse spaces
# =========================
def clean_text_unified(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.strip()
    # remove URLs
    text = re.sub(r"http\S+|www\S+|https\S+", " ", text)
    # remove zero-width
    text = remove_zero_width(text)
    # normalize fancy unicode to ascii
    text = normalize_unicode(text)
    # apply basic collapse of weird repeated punctuation
    text = re.sub(r'[\u2000-\u206F\u2E00-\u2E7F\p{P}]+', ' ', text) if False else text
    # first pass: merge purely spaced tokens
    text = merge_spaced_characters_simple(text)
    # second pass: merge tokens separated by punctuation/dots/underscores
    text = merge_obfuscated_sequences(text)
    # lowercase
    text = text.lower()
    # replace non alphanumeric with space
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    # collapse spaces
    text = re.sub(r"\s+", " ", text).strip()
    return text

# =========================
# LOAD, AUGMENT, CLEAN
# =========================
print("--- Loading data and preparing robust augmentations ---")
try:
    df_original = pd.read_csv(FILE_PATH)
    df_original = df_original.dropna(subset=[TEXT_COLUMN, TARGET_COLUMN])
    df_original[TARGET_COLUMN] = df_original[TARGET_COLUMN].astype(int)
except FileNotFoundError:
    print(f"ERROR: '{FILE_PATH}' not found. Aborting.")
    raise

# build synthetic base brands and phrases (wider variety)
prefixes = ['slot', 'jp', 'wd', 'depo', 'gacor', 'raja', 'bosku', 'mantap', 'pulau', 'vip', 'maxwin', 'king', 'boss']
numbers = [str(i) for i in range(1, 1000, 11)]
phrases = [
    'pasti jp di sini', 'wd cepat cair', 'deposito langsung', 'gacor banget', 
    'langsung wd tanpa ribet', 'buruan daftar', 'bonus untuk depo'
]

synthetic_rows = []
random.seed(SEED)
# create many synthetic base strings and many obfuscated variants
for i in range(200):  # more synthetic examples for robustness
    prefix = random.choice(prefixes)
    num = random.choice(numbers)
    brand = f"{prefix}{num}"
    phrase = random.choice(phrases)
    base_text = f"{phrase} main di {brand}"
    # create multiple obfuscated variants per base
    variants = generate_obfuscations(brand, n=8)
    for v in variants:
        # combine with phrase and sometimes with different markers
        form = random.choice([
            f"{phrase} {v}",
            f"buruan {v} jp",
            f"wd cepat di {v}",
            f"main di {v} aja",
            f"{v} gacor jp wd"
        ])
        cleaned = clean_text_unified(form)
        synthetic_rows.append({
            "comment_text": form,
            "cleaned_comment_text": cleaned,
            "target": 1,
            TEXT_COLUMN: cleaned
        })

df_synth = pd.DataFrame(synthetic_rows)

# combine with original but keep original cleaned if exists; otherwise clean
if 'cleaned_comment_text' not in df_original.columns:
    df_original['cleaned_comment_text'] = df_original[TEXT_COLUMN].astype(str).apply(clean_text_unified)
else:
    df_original[TEXT_COLUMN] = df_original[TEXT_COLUMN].astype(str).apply(clean_text_unified)
    df_original['cleaned_comment_text'] = df_original['cleaned_comment_text'].astype(str).apply(clean_text_unified)

# Combine
df_combined = pd.concat([df_original[[ 'comment_text', 'cleaned_comment_text', TARGET_COLUMN, TEXT_COLUMN ]], df_synth], ignore_index=True)
df_combined[TEXT_COLUMN] = df_combined[TEXT_COLUMN].fillna('').astype(str).apply(clean_text_unified)

# filter empties
df_combined = df_combined[df_combined[TEXT_COLUMN] != '']

print(f"Total rows after augmentation & cleaning: {len(df_combined)}")
print("Label distribution (overall):")
print(df_combined[TARGET_COLUMN].value_counts())

# =========================
# TRAIN-TEST SPLIT
# =========================
X = df_combined[TEXT_COLUMN]
Y = df_combined[TARGET_COLUMN].astype(int)

X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    X, Y, test_size=TEST_SIZE_FINAL, random_state=SEED, stratify=Y
)

# =========================
# TOKENIZER
# =========================
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text.astype(str))
with open(TOKENIZER_PATH, 'wb') as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)
print("Tokenizer saved.")

X_train_seq = tokenizer.texts_to_sequences(X_train_text.astype(str))
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')

X_test_seq = tokenizer.texts_to_sequences(X_test_text.astype(str))
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')

# =========================
# SMOTE (on padded sequences)
# =========================
print("Applying SMOTE to training data...")
smote = SMOTE(sampling_strategy='minority', random_state=SEED)
Y_train_np = Y_train.values
X_resampled, Y_resampled = smote.fit_resample(X_train_padded, Y_train_np)
X_train_final = np.array(X_resampled)
Y_train_final = np.array(Y_resampled)
print(f"After SMOTE: {len(X_train_final)} samples; Positives: {Y_train_final.sum()}")

# =========================
# MODEL: Bidirectional LSTM + Pooling
# =========================
def focal_loss(gamma=2., alpha=NEW_ALPHA):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        loss_value = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
                     - (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss_value)
    return loss

model = Sequential([
    Embedding(input_dim=MAX_WORDS, output_dim=EMBED_DIM),
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
    GlobalMaxPool1D(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

model.compile(loss=focal_loss(), optimizer=Adam(learning_rate=LR), metrics=['accuracy'])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=1)
]

print("Start training...")
history = model.fit(
    X_train_final, Y_train_final,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

print("Saving model...")
model.save(MODEL_PATH)
print(f"Model saved to: {MODEL_PATH}")

# =========================
# EVALUATION
# =========================
print("Evaluating on test set (pure real-world test set)...")
Y_test_np = Y_test.values
loss, acc = model.evaluate(X_test_padded, Y_test_np, verbose=0)
print(f"Test Loss: {loss:.4f}  Test Acc: {acc:.4f}")

y_pred_proba = model.predict(X_test_padded, verbose=0)
y_pred = (y_pred_proba > OPTIMAL_THRESHOLD).astype(int)
print("Classification report (threshold {:.2f}):".format(OPTIMAL_THRESHOLD))
print(classification_report(Y_test_np, y_pred))

# =========================
# PREDICTION FUNCTIONS (use same cleaning + tokenizer + model)
# =========================
def load_artifacts(model_path=MODEL_PATH, tokenizer_path=TOKENIZER_PATH):
    tk = None
    mdl = None
    if os.path.exists(tokenizer_path):
        with open(tokenizer_path, 'rb') as f:
            tk = pickle.load(f)
    if os.path.exists(model_path):
        mdl = load_model(model_path, custom_objects={'loss': focal_loss(alpha=NEW_ALPHA)})
    return mdl, tk

def predict_single(text: str, model_obj, tokenizer_obj, threshold=OPTIMAL_THRESHOLD):
    text_clean = clean_text_unified(text)
    seq = tokenizer_obj.texts_to_sequences([text_clean])
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    prob = model_obj.predict(pad, verbose=0)[0][0]
    label = 1 if prob > threshold else 0
    return {
        "input": text,
        "clean": text_clean,
        "prob": float(prob),
        "label": int(label)
    }

# Example quick test after training:
print("\nQuick manual checks:")
examples = [
    "W D 15 juta di B O S K U 7 7 7 langsung cair!",
    "p.u.l.a.u.w.i.n.8.8 jp wd cepat",
    "mantapjepe888 yuk main sini",
    "slotbaru555 gacor",
    "nonton film gratis di youtube channel ini saja"
]
mdl, tk = load_artifacts()
if mdl and tk:
    for ex in examples:
        r = predict_single(ex, mdl, tk)
        print(r)
else:
    print("Artifacts not found to run quick predict demo.")


--- Loading data and preparing robust augmentations ---
Total rows after augmentation & cleaning: 21773
Label distribution (overall):
target
0    18858
1     2915
Name: count, dtype: int64
Tokenizer saved.
Applying SMOTE to training data...
After SMOTE: 30172 samples; Positives: 15086
Start training...
Epoch 1/15
425/425 ━━━━━━━━━━━━━━━━━━━━ 151s 329ms/step - accuracy: 0.8668 - loss: 0.0396 - val_accuracy: 0.8376 - val_loss: 0.0480 - learning_rate: 0.0010
Epoch 2/15
425/425 ━━━━━━━━━━━━━━━━━━━━ 134s 314ms/step - accuracy: 0.9509 - loss: 0.0171 - val_accuracy: 0.9102 - val_loss: 0.0321 - learning_rate: 0.0010
Epoch 3/15
425/425 ━━━━━━━━━━━━━━━━━━━━ 134s 315ms/step - accuracy: 0.9785 - loss: 0.0076 - val_accuracy: 0.8694 - val_loss: 0.0800 - learning_rate: 0.0010
Epoch 4/15
425/425 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step - accuracy: 0.9884 - loss: 0.0046  
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
425/425 ━━━━━━━━━━━━━━━━━━━━ 136s 319ms/step - accuracy: 0.989

In [ ]:
# ============================================
# FILE: test_judol_model.py
# Standalone tester for robust judol model
# ============================================

import re
import pickle
import unicodedata
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# ============================================
# CONFIG
# ============================================
MODEL_PATH = "judol_detection_augmented_smote_robust.keras"
TOKENIZER_PATH = "tokenizer_augmented_robust.pickle"
MAX_LEN = 100
OPTIMAL_THRESHOLD = 0.65

# ============================================
# CLEANING PIPELINE (copy dari training file)
# ============================================

ZERO_WIDTH_CHARS = ['\u200b', '\u200c', '\u200d', '\u2060', '\uFEFF']
SEP_CHARS = r"[\s\.\-_\|•·\u2063]"
MIN_MERGE_LEN = 3

def remove_zero_width(text):
    for z in ZERO_WIDTH_CHARS:
        text = text.replace(z, '')
    return text

def normalize_unicode(text):
    text = unicodedata.normalize("NFKD", text)
    text = text.encode("ascii", "ignore").decode("utf-8", "ignore")
    return text

def merge_spaced_characters_simple(text, min_len=MIN_MERGE_LEN):
    tokens = text.split()
    merged, buffer = [], []
    for t in tokens:
        if len(t) == 1 and re.match(r'[A-Za-z0-9]', t):
            buffer.append(t)
        else:
            if len(buffer) >= min_len:
                merged.append("".join(buffer))
            elif buffer:
                merged.extend(buffer)
            buffer = []
            merged.append(t)
    if buffer:
        if len(buffer) >= min_len:
            merged.append("".join(buffer))
        else:
            merged.extend(buffer)
    return " ".join(merged)

def merge_obfuscated_sequences(text, min_len=MIN_MERGE_LEN):
    tokens = re.split(r'(\s+)', text)
    out, buffer = [], []
    for tok in tokens:
        if tok.isspace():
            if buffer:
                if len(buffer) >= min_len:
                    merged = re.sub(SEP_CHARS, "", "".join(buffer))
                    out.append(merged)
                else:
                    out.append("".join(buffer))
                buffer = []
            out.append(tok)
            continue
        if re.fullmatch(r'(?:[A-Za-z0-9]|' + SEP_CHARS + r')+$', tok):
            buffer.append(tok)
        else:
            if buffer:
                if len(buffer) >= min_len:
                    merged = re.sub(SEP_CHARS, "", "".join(buffer))
                    out.append(merged)
                else:
                    out.append("".join(buffer))
                buffer = []
            out.append(tok)
    if buffer:
        if len(buffer) >= min_len:
            out.append(re.sub(SEP_CHARS, "", "".join(buffer)))
        else:
            out.append("".join(buffer))
    return "".join(out)

def clean_text_unified(text):
    if not isinstance(text, str):
        text = str(text)

    text = re.sub(r"http\S+|www\S+|https\S+", " ", text)
    text = remove_zero_width(text)
    text = normalize_unicode(text)

    text = merge_spaced_characters_simple(text)
    text = merge_obfuscated_sequences(text)

    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()

    return text

# ============================================
# LOAD MODEL + TOKENIZER
# ============================================

print("Loading tokenizer...")
with open(TOKENIZER_PATH, "rb") as f:
    tokenizer = pickle.load(f)

print("Loading model...")
model = load_model(MODEL_PATH)

print("Artifacts loaded!")

# ============================================
# PREDICT FUNCTIONS
# ============================================

def predict(text):
    clean = clean_text_unified(text)
    seq = tokenizer.texts_to_sequences([clean])
    pad = pad_sequences(seq, maxlen=MAX_LEN)

    prob = float(model.predict(pad, verbose=0)[0][0])
    label = 1 if prob >= OPTIMAL_THRESHOLD else 0

    return {
        "input": text,
        "clean": clean,
        "prob": prob,
        "label": label
    }

# ============================================
# TEST EXAMPLES
# ============================================

tests = [
    "W D 15 juta di B O S K U 7 7 7 langsung cair!",
    "p.u.l.a.u.w.i.n.8.8 jp wd cepat",
    "mantapjepe888 yuk main sini",
    "slotbaru555 gacor",
    "nonton film gratis di youtube channel ini"
]

print("\n===== TEST RESULTS =====")
for t in tests:
    print(predict(t))


In [138]:
# file: robust_judol_training_and_prediction.py
# Robust model + augmentation + negative contrastive samples (fix false positive JP/WD)

import os, re, random, unicodedata, string, pickle
import numpy as np, pandas as pd, tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, GlobalMaxPool1D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# =========================
# CONFIG
# =========================
FILE_PATH = "final_production_judol_detection.csv"
MAX_WORDS = 30000
MAX_LEN = 100
EMBED_DIM = 128
BATCH_SIZE = 64
EPOCHS = 15
LR = 1e-3
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
TEST_SIZE_FINAL = 0.20
NEW_ALPHA = 0.55
OPTIMAL_THRESHOLD = 0.65
TEXT_COLUMN = 'combined_text'
TARGET_COLUMN = 'target'

TOKENIZER_PATH = "tokenizer_augmented_robust.pickle"
MODEL_PATH = "judol_detection_augmented_smote_robust.keras"

# =========================
# CLEANING HELPERS
# =========================
ZERO_WIDTH_CHARS = ['\u200b','\u200c','\u200d','\u2060','\uFEFF']
def remove_zero_width(t): 
    for z in ZERO_WIDTH_CHARS: t = t.replace(z,'')
    return t

def normalize_unicode(t):
    t = unicodedata.normalize('NFKD', t)
    return t.encode('ascii', 'ignore').decode('utf-8', 'ignore')

SEP_CHARS = r"[\s\.\-_\|•·\u2063]"
MIN_MERGE_LEN = 3

def merge_spaced_characters_simple(text, min_len=3):
    tokens = text.split()
    merged, buffer = [], []
    for t in tokens:
        if len(t)==1 and re.match(r'[A-Za-z0-9]', t):
            buffer.append(t)
        else:
            if len(buffer)>=min_len: merged.append("".join(buffer))
            else: merged.extend(buffer)
            buffer=[]
            merged.append(t)
    if buffer:
        if len(buffer)>=min_len: merged.append("".join(buffer))
        else: merged.extend(buffer)
    return " ".join(merged)

def merge_obfuscated_sequences(text, min_len=3):
    tokens = re.split(r'(\s+)', text)
    out, buffer = [], []
    for tok in tokens:
        if tok.isspace():
            if buffer:
                if len(buffer)>=min_len:
                    merged = re.sub(SEP_CHARS,'',"".join(buffer))
                    out.append(merged)
                else: out.append("".join(buffer))
                buffer=[]
            out.append(tok)
            continue
        if re.fullmatch(r'(?:[A-Za-z0-9]|'+SEP_CHARS+r')+$', tok):
            buffer.append(tok)
        else:
            if buffer:
                if len(buffer)>=min_len:
                    out.append(re.sub(SEP_CHARS,'',"".join(buffer)))
                else: out.append("".join(buffer))
                buffer=[]
            out.append(tok)
    if buffer:
        if len(buffer)>=min_len: out.append(re.sub(SEP_CHARS,'',"".join(buffer)))
        else: out.append("".join(buffer))
    return "".join(out)

def clean_text_unified(text):
    if not isinstance(text,str): text=str(text)
    text = re.sub(r"http\S+|www\S+|https\S+"," ",text)
    text = remove_zero_width(text)
    text = normalize_unicode(text)
    text = merge_spaced_characters_simple(text)
    text = merge_obfuscated_sequences(text)
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]"," ",text)
    text = re.sub(r"\s+"," ",text).strip()
    return text

# =========================
# DATASET LOAD
# =========================
print("--- Loading real dataset ---")
df_original = pd.read_csv(FILE_PATH)
df_original = df_original.dropna(subset=[TEXT_COLUMN, TARGET_COLUMN])
df_original[TARGET_COLUMN] = df_original[TARGET_COLUMN].astype(int)
df_original[TEXT_COLUMN] = df_original[TEXT_COLUMN].astype(str).apply(clean_text_unified)

# =========================
# SYNTHETIC POSITIVE (obfuscation)
# =========================
prefixes = ['slot','jp','wd','depo','gacor','raja','bosku','mantap','pulau','vip','maxwin','king','boss']
numbers = [str(i) for i in range(1, 500)]
phrases = [
    'pasti jp di sini','wd cepat cair','deposito langsung','gacor banget',
    'langsung wd tanpa ribet','buruan daftar','bonus untuk depo'
]

def generate_obfuscations(s):
    out=set()
    out.add(s); out.add(s.lower())
    def spaced(x): return " ".join(list(x))
    def dotted(x): return ".".join(list(x))
    def us(x): return "_".join(list(x))
    def leet(x): return x.translate(str.maketrans({'a':'4','o':'0','i':'1','e':'3','s':'5','t':'7'}))
    out.update({
        spaced(s), dotted(s), us(s), leet(s),
    })
    return list(out)[:6]

synthetic_rows=[]
for _ in range(1200):
    brand = random.choice(prefixes)+random.choice(numbers)
    phrase = random.choice(phrases)
    variants = generate_obfuscations(brand)
    for v in variants:
        form = random.choice([
            f"{phrase} {v}",
            f"buruan {v} jp",
            f"wd cepat di {v}",
            f"main di {v} aja",
            f"{v} gacor jp wd"
        ])
        cleaned = clean_text_unified(form)
        synthetic_rows.append({
            "comment_text": form,
            "cleaned_comment_text": cleaned,
            "target": 1,
            TEXT_COLUMN: cleaned
        })

df_synth = pd.DataFrame(synthetic_rows)

# =========================
# NEGATIVE CONTRASTIVE (fix false-positive JP/WD/DEPO)
# =========================
negative_phrases = [
    "akhirnya jp di turnamen badminton nasional",
    "tim itu berhasil jp di pertandingan final",
    "wd laptop saya ke service center",
    "wd foto ke harddisk sebelum format",
    "lagi depo uang ke rekening tabungan",
    "depo ke rekening perusahaan untuk bayar vendor",
    "dapet bonus dari kampus karena nilai bagus",
    "bonus akhir tahun dari kantor turun",
    "mantap banget videonya bang",
    "mantap penjelasannya dosen",
    "jp disini juara pertama bukan slot",
    "wd tugas kuliah dulu baru tidur",
    "depo file ke cloud biar aman",
    "bonus track lagu baru band itu keren"
]

neg_rows=[]
for txt in negative_phrases:
    c = clean_text_unified(txt)
    neg_rows.append({
        "comment_text": txt,
        "cleaned_comment_text": c,
        "target": 0,
        TEXT_COLUMN: c
    })

df_negative = pd.DataFrame(neg_rows)

# =========================
# COMBINE DATA
# =========================
df_combined = pd.concat([
    df_original[['comment_text','cleaned_comment_text','target',TEXT_COLUMN]],
    df_synth,
    df_negative
], ignore_index=True)

df_combined = df_combined[df_combined[TEXT_COLUMN]!=""]
print("Total samples:", len(df_combined))
print(df_combined['target'].value_counts())

# =========================
# TRAIN/TEST SPLIT
# =========================
X = df_combined[TEXT_COLUMN]
Y = df_combined['target'].astype(int)

X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    X, Y, test_size=TEST_SIZE_FINAL, random_state=SEED, stratify=Y
)

# =========================
# TOKENIZER
# =========================
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)
with open(TOKENIZER_PATH,'wb') as f: pickle.dump(tokenizer,f)

X_train_pad = pad_sequences(tokenizer.texts_to_sequences(X_train_text), maxlen=MAX_LEN)
X_test_pad = pad_sequences(tokenizer.texts_to_sequences(X_test_text), maxlen=MAX_LEN)

# =========================
# SMOTE
# =========================
sm = SMOTE()
X_res, Y_res = sm.fit_resample(X_train_pad, Y_train)

# =========================
# MODEL
# =========================
def focal_loss(gamma=2., alpha=NEW_ALPHA):
    def loss(y_true,y_pred):
        y_true=tf.cast(y_true,tf.float32)
        y_pred=tf.clip_by_value(tf.cast(y_pred,tf.float32),1e-7,1-1e-7)
        return tf.reduce_mean(
            -alpha*y_true*(1-y_pred)**gamma*tf.math.log(y_pred)
            -(1-alpha)*(1-y_true)*(y_pred**gamma)*tf.math.log(1-y_pred)
        )
    return loss

model = Sequential([
    Embedding(MAX_WORDS, EMBED_DIM),
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
    GlobalMaxPool1D(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    Dense(1, activation='sigmoid')
])

model.compile(loss=focal_loss(), optimizer=Adam(LR), metrics=['accuracy'])

model.fit(
    X_res, Y_res,
    validation_split=0.1,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[
        EarlyStopping(patience=4, restore_best_weights=True),
        ReduceLROnPlateau(patience=2, factor=0.5)
    ],
    verbose=1
)

model.save(MODEL_PATH)

# =========================
# EVALUATION
# =========================
pred_prob = model.predict(X_test_pad)
pred = (pred_prob>OPTIMAL_THRESHOLD).astype(int)

print(classification_report(Y_test, pred))

# =========================
# PREDICTION UTIL
# =========================
def load_artifacts(model_path=MODEL_PATH, tokenizer_path=TOKENIZER_PATH):
    with open(tokenizer_path,'rb') as f: tk = pickle.load(f)
    mdl = load_model(model_path, custom_objects={'loss':focal_loss()})
    return mdl, tk

def predict_single(text, mdl, tk, threshold=OPTIMAL_THRESHOLD):
    clean = clean_text_unified(text)
    seq = pad_sequences(tk.texts_to_sequences([clean]),maxlen=MAX_LEN)
    prob = mdl.predict(seq,verbose=0)[0][0]
    return {"input": text,"clean": clean,"prob": float(prob),"label": int(prob>threshold)}

print("\nQuick test:")
mdl, tk = load_artifacts()
tests = [
    "wd tugas kuliah dulu ya bang",
    "jp final badminton nasional",
    "p.u.l.a.u.w.i.n.8.8 gacor wd",
    "nonton film gratis di youtube"
]
for t in tests:
    print(predict_single(t, mdl, tk))


--- Loading real dataset ---
Total samples: 26135
target
0    18872
1     7263
Name: count, dtype: int64
Epoch 1/15
130/425 ━━━━━━━━━━━━━━━━━━━━ 1:38 333ms/step - accuracy: 0.7896 - loss: 0.0564

KeyboardInterrupt: 

In [132]:
# file: test_robust_judol_model.py

import os
import re
import unicodedata
import pickle
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# =========================
# CONFIG
# =========================
MAX_LEN = 100
NEW_ALPHA = 0.55
OPTIMAL_THRESHOLD = 0.65

TOKENIZER_PATH = "tokenizer_augmented_robust.pickle"
MODEL_PATH = "judol_detection_augmented_smote_robust.keras"

ZERO_WIDTH_CHARS = [
    '\u200b', '\u200c', '\u200d', '\u2060', '\uFEFF'
]

SEP_CHARS = r"[\s\.\-_\|•·\u2063]"
MIN_MERGE_LEN = 3


# =========================
# CLEANING PIPELINE (SAMA PERSIS DENGAN TRAINING)
# =========================

def remove_zero_width(text: str) -> str:
    for z in ZERO_WIDTH_CHARS:
        text = text.replace(z, '')
    return text

def normalize_unicode(text: str) -> str:
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

def merge_spaced_characters_simple(text: str, min_len=MIN_MERGE_LEN) -> str:
    tokens = text.split()
    merged = []
    buffer = []
    for t in tokens:
        if len(t) == 1 and re.match(r'[A-Za-z0-9]', t):
            buffer.append(t)
        else:
            if len(buffer) >= min_len:
                merged.append("".join(buffer))
            elif buffer:
                merged.extend(buffer)
            buffer = []
            merged.append(t)
    if buffer:
        if len(buffer) >= min_len:
            merged.append("".join(buffer))
        else:
            merged.extend(buffer)
    return " ".join(merged)

def merge_obfuscated_sequences(text: str, min_len=MIN_MERGE_LEN) -> str:
    tokens = re.split(r'(\s+)', text)
    out_tokens = []
    buffer = []
    for tok in tokens:
        if tok.isspace():
            if buffer:
                merged = re.sub(SEP_CHARS, '', "".join(buffer))
                out_tokens.append(merged)
                buffer = []
            out_tokens.append(tok)
            continue
        if re.fullmatch(r'(?:[A-Za-z0-9]|' + SEP_CHARS + r')+$', tok):
            buffer.append(tok)
        else:
            if buffer:
                merged = re.sub(SEP_CHARS, '', "".join(buffer))
                out_tokens.append(merged)
                buffer = []
            out_tokens.append(tok)
    if buffer:
        merged = re.sub(SEP_CHARS, '', "".join(buffer))
        out_tokens.append(merged)
    return "".join(out_tokens)

def clean_text_unified(text: str) -> str:
    if not isinstance(text, str):
        text = str(text)
    text = text.strip()
    text = re.sub(r"http\S+|www\S+|https\S+", " ", text)
    text = remove_zero_width(text)
    text = normalize_unicode(text)
    text = merge_spaced_characters_simple(text)
    text = merge_obfuscated_sequences(text)
    text = text.lower()
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text


# =========================
# LOAD ARTIFACTS
# =========================

def focal_loss(gamma=2., alpha=NEW_ALPHA):
    import tensorflow as tf
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        return -alpha * y_true * ((1 - y_pred) ** gamma) * tf.math.log(y_pred) - \
               (1 - alpha) * (1 - y_true) * (y_pred ** gamma) * tf.math.log(1 - y_pred)
    return loss

def load_artifacts():
    if not os.path.exists(TOKENIZER_PATH):
        raise FileNotFoundError(f"Tokenizer not found: {TOKENIZER_PATH}")

    with open(TOKENIZER_PATH, "rb") as f:
        tokenizer = pickle.load(f)

    if not os.path.exists(MODEL_PATH):
        raise FileNotFoundError(f"Model not found: {MODEL_PATH}")

    model = load_model(MODEL_PATH, custom_objects={"loss": focal_loss()})
    return model, tokenizer


# =========================
# PREDICT SINGLE / BATCH
# =========================

def predict_single(text, model, tokenizer, threshold=OPTIMAL_THRESHOLD):
    cleaned = clean_text_unified(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    prob = model.predict(pad, verbose=0)[0][0]
    label = int(prob > threshold)
    return {
        "input": text,
        "cleaned": cleaned,
        "prob": float(prob),
        "label": label
    }

def predict_batch(list_text, model, tokenizer, threshold=OPTIMAL_THRESHOLD):
    cleaned = [clean_text_unified(t) for t in list_text]
    seq = tokenizer.texts_to_sequences(cleaned)
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    probs = model.predict(pad, verbose=0).reshape(-1)
    labels = (probs > threshold).astype(int)
    result = []
    for raw, clean, prob, lbl in zip(list_text, cleaned, probs, labels):
        result.append({
            "input": raw,
            "cleaned": clean,
            "prob": float(prob),
            "label": int(lbl)
        })
    return result


# =========================
# CLI DEMO (run: python test_robust_judol_model.py)
# =========================

if __name__ == "__main__":
    model, tokenizer = load_artifacts()
    print("Artifacts loaded. Ready to test.\n")

    sample = [
        "Dajjal kalo liat cara fitnah ARMY ini.. pasti sungkem sujud sujud "
    ]

    print("=== EXAMPLE PREDICTIONS ===")
    out = predict_batch(sample, model, tokenizer)
    for r in out:
        print(r)


Artifacts loaded. Ready to test.

=== EXAMPLE PREDICTIONS ===
{'input': 'Dajjal kalo liat cara fitnah ARMY ini.. pasti sungkem sujud sujud ', 'cleaned': 'dajjal kalo liat cara fitnah army ini pasti sungkem sujud sujud', 'prob': 0.9538877010345459, 'label': 1}


# ya pakai clena text sama 

In [84]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import re
import unicodedata
import numpy as np

# ===============================================
# CONFIGURASI PENGUJIAN
# ===============================================
MODEL_PATH = "judol_detection_augmented_smote.h5"
TOKENIZER_PATH = "tokenizer_augmented.pickle"
MAX_LEN = 100             # Harus sama dengan MAX_LEN saat training
OPTIMAL_THRESHOLD = 0.65  # Threshold optimal yang Anda temukan

# ===============================================
# 1. DEFINISI FUNGSI CUSTOM (Focal Loss)
# ===============================================
# Focal Loss HARUS didefinisikan ulang agar Keras bisa memuat model
def focal_loss(gamma=2., alpha=0.55): # Alpha 0.8 harus sama dengan saat training
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        loss = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
                     - (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss)
    return loss

# ===============================================
# 2. FUNGSI PREPROCESSING UNTUK PREDIKSI (Sesuai Training)
# ===============================================
def clean_text_for_prediction(text):
    """Fungsi cleaning yang digunakan saat prediksi (dengan normalisasi Unicode yang lebih robust)"""
    if not isinstance(text, str):
        text = str(text)
        
    text = str(text).lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    # Normalisasi Unicode
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore') 
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# ===============================================
# 3. FUNGSI UTAMA PREDIKSI
# ===============================================
def predict_judol(text, model, tokenizer, threshold=OPTIMAL_THRESHOLD):
    """Memproses teks dan mengembalikan probabilitas dan label prediksi Judol."""
    
    text_clean = clean_text_for_prediction(text)
    
    # 3.1 Tokenisasi dan Padding
    seq = tokenizer.texts_to_sequences([text_clean])
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    
    # 3.2 Prediksi Model
    # model.predict mengembalikan array 2D [[probabilitas]]
    # Matikan verbose agar output bersih
    pred_prob = model.predict(pad, verbose=0)[0][0]
    is_judol = pred_prob > threshold
    
    label = "🔥 **JUDOL (SMOTE ML)**" if is_judol else "✅ Bukan Judol (SMOTE ML)"
    
    return label, pred_prob, text_clean

# ===============================================
# 4. MEMUAT MODEL DAN TOKENIZER
# ===============================================
try:
    # Memuat Model
    print(f"--- Memuat Model dari {MODEL_PATH} ---")
    model_loaded = load_model(MODEL_PATH, custom_objects={'loss': focal_loss(alpha=0.8)})
    
    # Memuat Tokenizer
    print(f"--- Memuat Tokenizer dari {TOKENIZER_PATH} ---")
    with open(TOKENIZER_PATH, 'rb') as handle:
        tokenizer_loaded = pickle.load(handle)
        
    print("\n✅ Model dan Tokenizer berhasil dimuat.")

except FileNotFoundError:
    print("❌ ERROR: File model atau tokenizer tidak ditemukan. Pastikan Anda sudah menjalankan script training.")
    exit()

# ===============================================
# 5. UJI COBA REAL-TIME
# ===============================================
print("\n=============================================")
print(f"🚀 UJI COBA MODEL DETEKSI JUDOL REAL-TIME (Threshold={OPTIMAL_THRESHOLD})")
print("=============================================")

test_comments = [
    # --- JUDOL TESTS ---
    "WD 500rb di GACOR777 mantap langsung cair!",
    "buruan daftar di MANTAPJP888, depo 100k langsung dapat bonus!",
    "main slot pasti untung, depo terus bosku",
    "BOSKU WD 5 JUTA HARI INI DI SITUS RAJA88.",
    "S L 0 T P R 1 M A 8 8 8. Jp 100k wd cepat.", 
    "Cepat gabung w1n k3lub, b4rus4n w d s4ld0 5j.",
    "Lagi gacor main di maxwin888 hari ini.",
    "Top banget! Terimakasih 𝗛𝗢𝗧𝗝𝗘𝗣𝗘𝟱𝟱 GOOD JOB!🀄🥰", # Uji Kasus Perbaikan
    "WD 1O jt di BOSSku777. *lgsg cair*!", 
    "Depo 100k, WD 20juta di RAJASLOT777. Pasti JP!",
    "Aku di MANTAP888. Gacor terus!", 
    "buruan daftar di MANTAPJEPE888 promonya gede banget", 
    "Paling the best! HOTJEPE55 Memang GOOD JOB banget!",
    "main di slotvip4d aja, depo 50 wd 500", 
    "gacorwd77 adalah situs paling oke.",
    "wd 1000 jp. link ada di bio ya.", 
    "jp 2OOm jt sudah di WD. Paling gacor di sini.", 
    "S L O T G A C O R 1 2 3 . C O M. Puji Tuhan, rejeki nomplok",
    "asli keren 𝗚𝗔𝗟𝗔𝗕𝗘𝗧𝟲𝟴𝟴 selalu bikin hepyy.", 
    "tokowin303 4w1 likeReply", 
    "Saya mau ngasih saran yg baik ya bg kemaren saya main di 𝙈𝘼𝙉𝙏𝙐𝙇𝟰𝟬𝟰 di kasih 22juta", 
    "𝗟𝗔𝗕𝗔𝟵𝟭 emang the real MVP, selalu bikin kita bahagia dengan kemenangan emang the real MVP, selalu bikin kita bahagia dengan kemenangan", 
    "slotvip777 depo 50 wd 500", 
    "JP boss 44D mantap",
    "💰 WD 5🟢0🟢0k di P U L A U W I N 88! Langsung 🤑 JP!", 
    "S L 0 T G A C 0 R 1 2 3 --- jp 500k wd cepat", 
    "𝗠𝗔𝗡𝗧𝗔𝗣𝗝𝗘𝗣𝗘𝟴𝟴🔥🔥 WD 2𝟬𝟬j", 
    "W D   1 0 j t .. d i 𝗕𝗢𝗦𝗦𝗞𝘂777 !!", 
    "jp besar 💵 WD kecil. Main di s l 0 t v 1 p 1 1 1 sekarang!", 
    "p_υ_l_α_υ_w_ι_n88 💰 jp 10J cepat WD!!", 
    "garudahoki🔥 mantapjepe888 🏆 jp 100k wd lancar", 
    "wd sepuluh juta di slotvip111 🎯 langsung cair", 
    "wd cepat + JP 200k main di RAJASLOT777 sekarang", 
    "Daft4r di S G 1 ‧ 8‧8‧ bentar lagi ada drop mantap.",
    "mAin di b𝚘𝚜ﻜ𝚞7𝟽𝟽 enak bet, wd aman.",
    "bo.s ku. tujuh tujuh tujuh lagi gila2an promo wdee.",
    "m​a​i​n​ ​d​i​ ​b​o​s​k​u​7​7​7​ g​a​s​k​e​n​.",
    "Coba deh di \"QlRTT1QxMDA=\" katanya jam 3 pagi gacor.",
    "Main di cptlv888 lumayan tarikannya.",
    "我 在 博 司 苦 七星 七星 七星 刚 取 款.",
    "b0sku77? gas bet wd ngebut bgt.",
    "b o s k u\n7 7 7\n...disusun vertikal.",
    "WD 600 tadi, no cap. That site hits diff fr.",
    "M@!N di B0$KU^777 cuy wd kuzz teroosss.",
    "bo sku 7 7 7 — karena kamera burem jd gini.",

    # --- NON-JUDOL TESTS ---
    "Beli skin game baru ah, keren nih!", 
    "Ini channel YouTube favoritku, sering update video baru.",
    "Makan malam apa hari ini?",
    "Saya mau nonton film gratis di y0utube, tapi juga WD 5 ribu dari tabungan",
    "WD 50 ribu saja, buat beli kopi.", 
    "wah mantap nih, motor baru udah di WD sama pemiliknya", 
    "itu tempat bosku yang warna hijau itu loh, yg sering iklan jam 2 pagi.",
    "Tadi malem aku pecah telur di tempat sebelah. Tarikan bersih bgt.", 
    "Udah, masuk aja ke yang kemarin aku bilang. Yang tiap hari bagi hadiah itu.",
    "Wkwk mantap jp lagi di “platform terpercaya” itu katanya 🤡",
    "bos ku sudah datang tadi pagi bawa hadiah 25k. Gila.",
    "Ingat, keberanian itu modal. Kalau mau hasil besar, tempat itu udah aku rekomendasiin kemarin.",
    "Semoga sukses selalu dan sehat terus ya. Kontennya sangat bermanfaat!", 
    "Saya transfer 500 ribu ke rekening teman tadi pagi, semoga lancar.", 
    "Main Mobile Legends seru banget, tapi harus top up dulu buat beli diamond.", 
    "Saya pesan nasi Padang di warung Bossku tadi, rasanya mantap.", 
    "Saya sudah JP (Jelas Puas) sama pelayanan toko ini.", 
    "Komentar ini hanya berisi pujian, tidak ada unsur promosi atau judi.",
    "link untuk download materi kuliah ada di deskripsi video ya teman-teman.", 
    "Terima kasih atas video tutorial cara WD (Withdrawal) uang dari platform saham." ,
    "Dijamin happy udah kaya rumah kedua 𝐀𝐑𝐄𝐍𝐀𝟗𝟐𝟑 disini 👏",
    "cuma di tempat lu bang yang paling amanah🙌",
    "𝙈𝙀𝙂𝘼𝘽𝙀𝙏𝟭𝟴𝟵 emang the real MVP selalu bikin kita bahagia dengan kemenangannya",
    # Non-Judol menggunakan WD (Withdraw) dan Nominal
    "Tadi pagi aku WD (Withdraw) uang tunai dari ATM 500 ribu.",
    "WD dana beasiswa sudah cair, total 5 juta! Alhamdulillah.",
    "Jangan lupa WD hasil penjualan dari platform e-commerce itu ya.",
    
    # Non-Judol menggunakan JP (Jelas Puas, Jaminan Produk, etc.)
    "Pelayanan toko ini JP (Jelas Puas) banget, barangnya 'Gacor'!",
    "Aku JP (Jaminan Produk) 100% kalau kamu beli di Bossku Official Store.",
    "Wah, si Bossku tadi pagi datang bawa hadiah, mantap JP!",

    # Non-Judol menggunakan Gacor, Bossku, Depo (konteks non-judi)
    "Motor baruku Gacor banget, tarikannya mantap!",
    "Si Bossku (panggilan teman) tadi Depo (Deposit) makanan ke meja.",
    "Restoran ini Gacor tiap hari, ramenya nggak ketulungan.",
    
    # Non-Judol dengan Kombinasi Kata Kunci Ambigu
    "Aku dapat promo Depo 50k, dapat 50k di Toko X (bukan situs judi).",
    "Gacor banget, main game Mobile Legends dapat JP (Juara Pertama).",
    "Kata Bossku, ini adalah situs terbaik untuk beli akun game (bukan judi).",
    "WD 5 ribu aja buat beli kopi, soalnya dompet udah 'Gacor'." # Gacor = Kosong
]

for comment in test_comments:
    # Memastikan model tidak menampilkan pesan saat memprediksi
    label, prob, clean_text = predict_judol(comment, model_loaded, tokenizer_loaded, OPTIMAL_THRESHOLD)
    
    print(f"\n💬 Input: '{comment}'")
    print(f"🧹 Clean: '{clean_text}'")
    print(f"🎯 Prediksi: {label} (Probabilitas: {prob:.4f})")

print("\n=============================================")

--- Memuat Model dari judol_detection_augmented_smote.h5 ---


--- Memuat Tokenizer dari tokenizer_augmented.pickle ---

✅ Model dan Tokenizer berhasil dimuat.

🚀 UJI COBA MODEL DETEKSI JUDOL REAL-TIME (Threshold=0.65)

💬 Input: 'WD 500rb di GACOR777 mantap langsung cair!'
🧹 Clean: 'wd 500rb di gacor777 mantap langsung cair'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.7679)

💬 Input: 'buruan daftar di MANTAPJP888, depo 100k langsung dapat bonus!'
🧹 Clean: 'buruan daftar di mantapjp888 depo 100k langsung dapat bonus'
🎯 Prediksi: ✅ Bukan Judol (SMOTE ML) (Probabilitas: 0.5325)

💬 Input: 'main slot pasti untung, depo terus bosku'
🧹 Clean: 'main slot pasti untung depo terus bosku'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.7273)

💬 Input: 'BOSKU WD 5 JUTA HARI INI DI SITUS RAJA88.'
🧹 Clean: 'bosku wd 5 juta hari ini di situs raja88'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.8439)

💬 Input: 'S L 0 T P R 1 M A 8 8 8. Jp 100k wd cepat.'
🧹 Clean: 's l 0 t p r 1 m a 8 8 8 jp 100k wd cepat'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Prob

In [85]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import re
import unicodedata
import numpy as np

# ===============================================
# CONFIGURASI PENGUJIAN
# ===============================================
MODEL_PATH = "judol_detection_augmented_smote.h5"
TOKENIZER_PATH = "tokenizer_augmented.pickle"
MAX_LEN = 100             # Harus sama dengan MAX_LEN saat training
OPTIMAL_THRESHOLD = 0.65  # Threshold optimal yang Anda temukan

# ===============================================
# 1. DEFINISI FUNGSI CUSTOM (Focal Loss)
# ===============================================
# Focal Loss HARUS didefinisikan ulang agar Keras bisa memuat model
def focal_loss(gamma=2., alpha=0.55): # Alpha 0.8 harus sama dengan saat training
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        loss = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
                     - (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss)
    return loss

# ===============================================
# 2. FUNGSI PREPROCESSING UNTUK PREDIKSI (Sesuai Training)
# ===============================================
def clean_text_for_prediction(text):
    """Fungsi cleaning yang digunakan saat prediksi (dengan normalisasi Unicode yang lebih robust)"""
    if not isinstance(text, str):
        text = str(text)
        
    # 1. Hapus URL
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    
    # NFKD: Mengubah karakter fancy (misal 𝗛𝗢𝗧𝐉𝐄𝐏𝐄𝟱𝟱) menjadi bentuk dasarnya (HOTJEPE55)
    text = unicodedata.normalize('NFKD', text)
    
    # Encode/Decode: Menghapus semua karakter non-ASCII (seperti emoji 🀄🥰, diakritik)
    # yang tersisa setelah NFKD, hanya menyisakan karakter dasar ASCII.
    # Ini adalah langkah kunci untuk mengatasi masalah hilangnya karakter fancy.
    text = text.encode('ascii', 'ignore').decode('utf-8')
    
    # 3. Lowercase (Dapat dilakukan setelah konversi ASCII)
    text = text.lower()
    
    # 4. Hapus sisa karakter yang bukan a-z, 0-9, atau spasi
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    
    # 5. Rapikan spasi
    text = re.sub(r"\s+", " ", text).strip()
    return text

# ===============================================
# 3. FUNGSI UTAMA PREDIKSI
# ===============================================
def predict_judol(text, model, tokenizer, threshold=OPTIMAL_THRESHOLD):
    """Memproses teks dan mengembalikan probabilitas dan label prediksi Judol."""
    
    text_clean = clean_text_for_prediction(text)
    
    # 3.1 Tokenisasi dan Padding
    seq = tokenizer.texts_to_sequences([text_clean])
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    
    # 3.2 Prediksi Model
    # model.predict mengembalikan array 2D [[probabilitas]]
    # Matikan verbose agar output bersih
    pred_prob = model.predict(pad, verbose=0)[0][0]
    is_judol = pred_prob > threshold
    
    label = "🔥 **JUDOL (SMOTE ML)**" if is_judol else "✅ Bukan Judol (SMOTE ML)"
    
    return label, pred_prob, text_clean

# ===============================================
# 4. MEMUAT MODEL DAN TOKENIZER
# ===============================================
try:
    # Memuat Model
    print(f"--- Memuat Model dari {MODEL_PATH} ---")
    model_loaded = load_model(MODEL_PATH, custom_objects={'loss': focal_loss(alpha=0.8)})
    
    # Memuat Tokenizer
    print(f"--- Memuat Tokenizer dari {TOKENIZER_PATH} ---")
    with open(TOKENIZER_PATH, 'rb') as handle:
        tokenizer_loaded = pickle.load(handle)
        
    print("\n✅ Model dan Tokenizer berhasil dimuat.")

except FileNotFoundError:
    print("❌ ERROR: File model atau tokenizer tidak ditemukan. Pastikan Anda sudah menjalankan script training.")
    exit()

# ===============================================
# 5. UJI COBA REAL-TIME
# ===============================================
print("\n=============================================")
print(f"🚀 UJI COBA MODEL DETEKSI JUDOL REAL-TIME (Threshold={OPTIMAL_THRESHOLD})")
print("=============================================")

test_comments = [
    # --- JUDOL TESTS ---
    "WD 500rb di GACOR777 mantap langsung cair!",
    "buruan daftar di MANTAPJP888, depo 100k langsung dapat bonus!",
    "main slot pasti untung, depo terus bosku",
    "BOSKU WD 5 JUTA HARI INI DI SITUS RAJA88.",
    "S L 0 T P R 1 M A 8 8 8. Jp 100k wd cepat.", 
    "Cepat gabung w1n k3lub, b4rus4n w d s4ld0 5j.",
    "Lagi gacor main di maxwin888 hari ini.",
    "Top banget! Terimakasih 𝗛𝗢𝗧𝗝𝗘𝗣𝗘𝟱𝟱 GOOD JOB!🀄🥰", # Uji Kasus Perbaikan
    "WD 1O jt di BOSSku777. *lgsg cair*!", 
    "Depo 100k, WD 20juta di RAJASLOT777. Pasti JP!",
    "Aku di MANTAP888. Gacor terus!", 
    "buruan daftar di MANTAPJEPE888 promonya gede banget", 
    "Paling the best! HOTJEPE55 Memang GOOD JOB banget!",
    "main di slotvip4d aja, depo 50 wd 500", 
    "gacorwd77 adalah situs paling oke.",
    "wd 1000 jp. link ada di bio ya.", 
    "jp 2OOm jt sudah di WD. Paling gacor di sini.", 
    "S L O T G A C O R 1 2 3 . C O M. Puji Tuhan, rejeki nomplok",
    "asli keren 𝗚𝗔𝗟𝗔𝗕𝗘𝗧𝟲𝟴𝟴 selalu bikin hepyy.", 
    "tokowin303 4w1 likeReply", 
    "Saya mau ngasih saran yg baik ya bg kemaren saya main di 𝙈𝘼𝙉𝙏𝙐𝙇𝟰𝟬𝟰 di kasih 22juta", 
    "𝗟𝗔𝗕𝗔𝟵𝟭 emang the real MVP, selalu bikin kita bahagia dengan kemenangan emang the real MVP, selalu bikin kita bahagia dengan kemenangan", 
    "slotvip777 depo 50 wd 500", 
    "JP boss 44D mantap",
    "💰 WD 5🟢0🟢0k di PULAUWIN 88! Langsung 🤑 JP!", 
    "S L 0 T G A C 0 R 1 2 3 --- jp 500k wd cepat", 
    "𝗠𝗔𝗡𝗧𝗔𝗣𝗝𝗘𝗣𝗘𝟴𝟴🔥🔥 WD 2𝟬𝟬j", 
    "W D   1 0 j t .. d i 𝗕𝗢𝗦𝗦𝗞𝘂777 !!", 
    "jp besar 💵 WD kecil. Main di s l 0 t v 1 p 1 1 1 sekarang!", 
    "p_υ_l_α_υ_w_ι_n88 💰 jp 10J cepat WD!!", 
    "garudahoki🔥 mantapjepe888 🏆 jp 100k wd lancar", 
    "wd sepuluh juta di slotvip111 🎯 langsung cair", 
    "wd cepat + JP 200k main di RAJASLOT777 sekarang", 
    "Daft4r di S G 1 ‧ 8‧8‧ bentar lagi ada drop mantap.",
    "mAin di b𝚘𝚜ﻜ𝚞7𝟽𝟽 enak bet, wd aman.",
    "bo.s ku. tujuh tujuh tujuh lagi gila2an promo wdee.",
    "m​a​i​n​ ​d​i​ ​b​o​s​k​u​7​7​7​ g​a​s​k​e​n​.",
    "Coba deh di \"QlRTT1QxMDA=\" katanya jam 3 pagi gacor.",
    "Main di cptlv888 lumayan tarikannya.",
    "我 在 博 司 苦 七星 七星 七星 刚 取 款.",
    "b0sku77? gas bet wd ngebut bgt.",
    "b o s k u\n7 7 7\n...disusun vertikal.",
    "WD 600 tadi, no cap. That site hits diff fr.",
    "M@!N di B0$KU^777 cuy wd kuzz teroosss.",
    "bo sku 7 7 7 — karena kamera burem jd gini.",

    # --- NON-JUDOL TESTS ---
    "Beli skin game baru ah, keren nih!", 
    "Ini channel YouTube favoritku, sering update video baru.",
    "Makan malam apa hari ini?",
    "Saya mau nonton film gratis di y0utube, tapi juga WD 5 ribu dari tabungan",
    "WD 50 ribu saja, buat beli kopi.", 
    "wah mantap nih, motor baru udah di WD sama pemiliknya", 
    "itu tempat bosku yang warna hijau itu loh, yg sering iklan jam 2 pagi.",
    "Tadi malem aku pecah telur di tempat sebelah. Tarikan bersih bgt.", 
    "Udah, masuk aja ke yang kemarin aku bilang. Yang tiap hari bagi hadiah itu.",
    "Wkwk mantap jp lagi di “platform terpercaya” itu katanya 🤡",
    "bos ku sudah datang tadi pagi bawa hadiah 25k. Gila.",
    "Ingat, keberanian itu modal. Kalau mau hasil besar, tempat itu udah aku rekomendasiin kemarin.",
    "Semoga sukses selalu dan sehat terus ya. Kontennya sangat bermanfaat!", 
    "Saya transfer 500 ribu ke rekening teman tadi pagi, semoga lancar.", 
    "Main Mobile Legends seru banget, tapi harus top up dulu buat beli diamond.", 
    "Saya pesan nasi Padang di warung Bossku tadi, rasanya mantap.", 
    "Saya sudah JP (Jelas Puas) sama pelayanan toko ini.", 
    "Komentar ini hanya berisi pujian, tidak ada unsur promosi atau judi.",
    "link untuk download materi kuliah ada di deskripsi video ya teman-teman.", 
    "Terima kasih atas video tutorial cara WD (Withdrawal) uang dari platform saham." ,
    "Dijamin happy udah kaya rumah kedua 𝐀𝐑𝐄𝐍𝐀𝟗𝟐𝟑 disini 👏",
    "cuma di tempat lu bang yang paling amanah🙌",
    "𝙈𝙀𝙂𝘼𝘽𝙀𝙏𝟭𝟴𝟵 emang the real MVP selalu bikin kita bahagia dengan kemenangannya",
    # Non-Judol menggunakan WD (Withdraw) dan Nominal
    "Tadi pagi aku WD (Withdraw) uang tunai dari ATM 500 ribu.",
    "WD dana beasiswa sudah cair, total 5 juta! Alhamdulillah.",
    "Jangan lupa WD hasil penjualan dari platform e-commerce itu ya.",
    
    # Non-Judol menggunakan JP (Jelas Puas, Jaminan Produk, etc.)
    "Pelayanan toko ini JP (Jelas Puas) banget, barangnya 'Gacor'!",
    "Aku JP (Jaminan Produk) 100% kalau kamu beli di Bossku Official Store.",
    "Wah, si Bossku tadi pagi datang bawa hadiah, mantap JP!",

    # Non-Judol menggunakan Gacor, Bossku, Depo (konteks non-judi)
    "Motor baruku Gacor banget, tarikannya mantap!",
    "Si Bossku (panggilan teman) tadi Depo (Deposit) makanan ke meja.",
    "Restoran ini Gacor tiap hari, ramenya nggak ketulungan.",
    
    # Non-Judol dengan Kombinasi Kata Kunci Ambigu
    "Aku dapat promo Depo 50k, dapat 50k di Toko X (bukan situs judi).",
    "Gacor banget, main game Mobile Legends dapat JP (Juara Pertama).",
    "Kata Bossku, ini adalah situs terbaik untuk beli akun game (bukan judi).",
    "WD 5 ribu aja buat beli kopi, soalnya dompet udah 'Gacor'." # Gacor = Kosong
]

for comment in test_comments:
    # Memastikan model tidak menampilkan pesan saat memprediksi
    label, prob, clean_text = predict_judol(comment, model_loaded, tokenizer_loaded, OPTIMAL_THRESHOLD)
    
    print(f"\n💬 Input: '{comment}'")
    print(f"🧹 Clean: '{clean_text}'")
    print(f"🎯 Prediksi: {label} (Probabilitas: {prob:.4f})")

print("\n=============================================")

--- Memuat Model dari judol_detection_augmented_smote.h5 ---


--- Memuat Tokenizer dari tokenizer_augmented.pickle ---

✅ Model dan Tokenizer berhasil dimuat.

🚀 UJI COBA MODEL DETEKSI JUDOL REAL-TIME (Threshold=0.65)

💬 Input: 'WD 500rb di GACOR777 mantap langsung cair!'
🧹 Clean: 'wd 500rb di gacor777 mantap langsung cair'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.7679)

💬 Input: 'buruan daftar di MANTAPJP888, depo 100k langsung dapat bonus!'
🧹 Clean: 'buruan daftar di mantapjp888 depo 100k langsung dapat bonus'
🎯 Prediksi: ✅ Bukan Judol (SMOTE ML) (Probabilitas: 0.5325)

💬 Input: 'main slot pasti untung, depo terus bosku'
🧹 Clean: 'main slot pasti untung depo terus bosku'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.7273)

💬 Input: 'BOSKU WD 5 JUTA HARI INI DI SITUS RAJA88.'
🧹 Clean: 'bosku wd 5 juta hari ini di situs raja88'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Probabilitas: 0.8439)

💬 Input: 'S L 0 T P R 1 M A 8 8 8. Jp 100k wd cepat.'
🧹 Clean: 's l 0 t p r 1 m a 8 8 8 jp 100k wd cepat'
🎯 Prediksi: 🔥 **JUDOL (SMOTE ML)** (Prob

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import re
import unicodedata
import numpy as np
import sys

# --- MEMBUTUHKAN SASTRAWI UNTUK KONSISTENSI PREPROCESSING ---
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# ===============================================
# CONFIGURASI PENGUJIAN
# ===============================================
MODEL_PATH = "judol_detection_augmented_smote.h5"
TOKENIZER_PATH = "tokenizer_augmented.pickle"
MAX_LEN = 100             # Harus sama dengan MAX_LEN saat training
OPTIMAL_THRESHOLD = 0.65  # Kembali ke threshold optimal awal (atau disesuaikan nanti)
TRAINING_ALPHA = 0.55     # Alpha Focal Loss dari training

# ===============================================
# 0. INISIALISASI SASTRAWI
# ===============================================
# Inisialisasi Sastrawi harus SAMA persis dengan yang digunakan saat training
try:
    factory = StopWordRemoverFactory()
    stopword_remover = factory.create_stop_word_remover()

    factory_stemmer = StemmerFactory()
    stemmer = factory_stemmer.create_stemmer()
except Exception as e:
    print(f"❌ ERROR: Gagal inisialisasi Sastrawi. Pastikan Sastrawi sudah terinstall.")
    print(f"Detail error: {e}")
    sys.exit()


# ===============================================
# 1. DEFINISI FUNGSI CUSTOM (Focal Loss)
# ===============================================
def focal_loss(gamma=2., alpha=TRAINING_ALPHA): 
    """Fungsi Focal Loss yang dibutuhkan untuk memuat model Keras."""
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        loss = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
                     - (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss)
    return loss

# ===============================================
# 2. FUNGSI PREPROCESSING KONSISTEN (Menggabungkan semua langkah training)
# ===============================================
def clean_text_for_prediction(text, stopword_remover, stemmer):
    """
    Fungsi cleaning yang MENCERMINKAN SECARA TEPAT langkah-langkah dalam text preprocessing.py
    (tanpa fungsi remove_text_decorations yang tidak diberikan, diasumsikan sudah ada di langkah awal).
    """
    if not isinstance(text, str):
        text = str(text)

    # 1. Normalisasi Unicode (NFKD) & Konversi ke ASCII (Langkah dari skrip lama, tetap dipertahankan)
    # Ini membantu mengatasi font-font unik seperti pada HOTJEPE55
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    
    # 2. Hapus URL (Langkah dari skrip lama, tetap dipertahankan)
    text = re.sub(r"http\S+|www\S+|https\S+", "", text)
    
    # 3. Lowercase (DILAKUKAN SEBELUM STOPWORD/STEMMING)
    text = text.lower()
    
    # 4. Hapus karakter yang bukan a-z, 0-9, atau spasi (sesuai langkah 7 di training)
    # Di sini kita hanya menyisakan alfanumerik dan spasi agar konsisten sebelum Sastrawi
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    
    # 5. Rapikan spasi dan trim (Langkah dari skrip lama, tetap dipertahankan)
    text = re.sub(r"\s+", " ", text).strip()
    
    # --- LANGKAH PENTING: STOPWORD REMOVAL (Sastrawi) ---
    text = stopword_remover.remove(text)
    
    # --- LANGKAH PENTING: STEMMING (Sastrawi) ---
    text = stemmer.stem(text)

    # 6. Rapikan spasi lagi setelah stemming/stopword
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

# ===============================================
# 3. FUNGSI UTAMA PREDIKSI
# ===============================================
def predict_judol(text, model, tokenizer, threshold, stopword_remover, stemmer):
    """Memproses teks dan mengembalikan probabilitas dan label prediksi Judol."""
    
    # Meneruskan objek Sastrawi ke fungsi cleaning
    text_clean = clean_text_for_prediction(text, stopword_remover, stemmer)
    
    # 3.1 Tokenisasi dan Padding
    seq = tokenizer.texts_to_sequences([text_clean])
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    
    # 3.2 Prediksi Model
    if pad.size == 0:
        pred_prob = 0.0
    else:
        pred_prob = model.predict(pad, verbose=0)[0][0]
        
    is_judol = pred_prob > threshold
    
    label = f"🔥 **JUDOL (SMOTE ML)** (Prob > {threshold})" if is_judol else f"✅ Bukan Judol (Prob <= {threshold})"
    
    return label, pred_prob, text_clean

# ===============================================
# 4. MEMUAT MODEL DAN TOKENIZER
# ===============================================
try:
    print(f"--- Memuat Model dari {MODEL_PATH} (Alpha={TRAINING_ALPHA}) ---")
    model_loaded = load_model(
        MODEL_PATH, 
        custom_objects={'loss': focal_loss(alpha=TRAINING_ALPHA)} 
    )
    
    print(f"--- Memuat Tokenizer dari {TOKENIZER_PATH} ---")
    with open(TOKENIZER_PATH, 'rb') as handle:
        tokenizer_loaded = pickle.load(handle)
        
    print("\n✅ Model dan Tokenizer berhasil dimuat.")

except FileNotFoundError:
    print(f"❌ ERROR: File model ({MODEL_PATH}) atau tokenizer ({TOKENIZER_PATH}) tidak ditemukan.")
    sys.exit()
except Exception as e:
    print(f"❌ ERROR: Gagal memuat model atau tokenizer: {e}")
    sys.exit()

# ===============================================
# 5. UJI COBA REAL-TIME DATASET
# ===============================================
print("\n=============================================")
print(f"🚀 UJI COBA MODEL DETEKSI JUDOL KONSISTEN (Threshold={OPTIMAL_THRESHOLD})")
print("=============================================")

test_comments = [
    # --- JUDOL TESTS (Target: TP) ---
    "WD 500rb di GACOR777 mantap langsung cair!", # TP
    "buruan daftar di MANTAPJP888, depo 100k langsung dapat bonus!", # TP
    "jp besar 💵 WD kecil. Main di s l 0 t v 1 p 1 1 1 sekarang!", # TP
    "WD 1O jt di BOSSku777. *lgsg cair*!", # TP
    "Tokowin303 4w1 likeReply", # TP
    
    # --- NON-JUDOL TESTS (Target: TN - Fokus Perbaikan FP) ---
    "Semoga sukses selalu dan sehat terus ya. Kontennya sangat bermanfaat!", # FP sebelumnya 0.9704 -> Target: TN
    "Komentar ini hanya berisi pujian, tidak ada unsur promosi atau judi.", # FP sebelumnya 0.9724 -> Target: TN
    "WD dana beasiswa sudah cair, total 5 juta! Alhamdulillah.", # FP sebelumnya 0.9466 -> Target: TN
    "Terima kasih atas video tutorial cara WD (Withdrawal) uang dari platform saham.", # FP sebelumnya 0.9527 -> Target: TN
    "Pelayanan toko ini JP (Jelas Puas) banget, barangnya 'Gacor'!", # FP sebelumnya 0.9426 -> Target: TN
    "Aku dapat promo Depo 50k, dapat 50k di Toko X (bukan situs judi).", # TN
    "Main Mobile Legends seru banget, tapi harus top up dulu buat beli diamond.", # TN
    "WD 50 ribu saja, buat beli kopi." # TN
]

for comment in test_comments:
    # Menggunakan fungsi prediksi yang konsisten
    label, prob, clean_text = predict_judol(comment, model_loaded, tokenizer_loaded, OPTIMAL_THRESHOLD, stopword_remover, stemmer)
    
    print(f"\n💬 Input: '{comment}'")
    print(f"🧹 Clean: '{clean_text}'")
    print(f"🎯 Prediksi: {label} (Probabilitas: {prob:.4f})")

print("\n=============================================")
print("Uji coba selesai.")
print("Perhatikan kolom 'Clean' sekarang. Teks harusnya lebih pendek karena stopword sudah hilang dan kata-kata sudah di-stem.")
print("Jika False Positive (FP) yang confidence-nya tinggi (seperti 'Semoga sukses...') kini menjadi True Negative (TN), berarti konsistensi preprocessing adalah solusinya.")

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# --- MODUL WAJIB ---
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import unicodedata
import pickle
import sys
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
# Modul Sastrawi Wajib
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# Modul Tambahan untuk Progress Bar
from tqdm import tqdm
tqdm.pandas()


# ===============================================
# CONFIGURASI UTAMA & INISIALISASI SASTRAWI
# ===============================================
FILE_PATH = "final_production_judol_detection.csv"
MAX_WORDS = 20000
MAX_LEN = 100
EMBED_DIM = 128
BATCH_SIZE = 64
EPOCHS = 25
LR = 1e-3
SEED = 42
NEW_ALPHA = 0.55
OPTIMAL_THRESHOLD = 0.65
TEXT_COLUMN = 'cleaned_comment_text'
TARGET_COLUMN = 'target'
TEST_SIZE_FINAL = 0.20
NEW_MODEL_AUG_PATH = "judol_detection_augmented_smote.keras" # <--- PERUBAHAN UTAMA: Gunakan ekstensi .keras

np.random.seed(SEED)
tf.random.set_seed(SEED)

# INISIALISASI SASTRAWI
factory = StopWordRemoverFactory()
stopword_remover = factory.create_stop_word_remover()
factory_stemmer = StemmerFactory()
stemmer = factory_stemmer.create_stemmer()

# ===============================================
# 1. DATA AUGMENTASI: SINTESIS JUDOL (LABEL 1)
# ===============================================
prefixes = ['slot', 'jp', 'wd', 'depo', 'gacor', 'raja', 'bosku', 'mantap']
numbers = [str(i) for i in range(100, 999, 100)] + [str(i) for i in range(11, 99, 11)]
phrases = ['gacor banget di sini!', 'pasti jp di situs ini', 'wd cepat tanpa ribet', 'cair terus bosku']

synthetic_data_pos = []
for i in range(200):
    prefix = np.random.choice(prefixes)
    num = np.random.choice(numbers)
    phrase = np.random.choice(phrases)
    brand_name = prefix + num
    comment = f"{phrase} coba main di {brand_name} aja deh"
    synthetic_data_pos.append({"comment_text": comment, "target": 1})

df_synthetic_pos = pd.DataFrame(synthetic_data_pos)

# ===============================================
# 2. DATA AUGMENTASI: SINTESIS NON-JUDOL (LABEL 0)
# ===============================================
NEGATIVE_AUGMENTATION_DATA = [
    "WD uang beasiswa sudah cair 5 juta alhamdulillah",
    "WD dana dari platform saham tutorialnya jelas",
    "WD gaji dari kantor total 10 juta",
    "WD uang tunai 500 ribu dari ATM",
    "Withdrawal dana P2P lending cair ke rekening",
    "tarik uang 1 juta untuk beli buku kuliah",
    "gacor banget motor tarikan mantap",
    "gacor banget pelayanan toko ini",
    "jp jelas puas sama produknya",
    "motor baru ini gacor sekali",
    "mobil jerman ini tarikannya mantap",
    "jp (jelas puas) sama hasilnya",
    "bosku beli kopi di warung",
    "saya sudah deposit uang di bank",
    "depo uang kuliah semester 3",
    "akun ini sudah top up diamond",
    "kami adalah agen perjalanan resmi",
]

df_neg_aug = pd.DataFrame({'comment_text': NEGATIVE_AUGMENTATION_DATA, 'target': 0})

print(f"✅ {len(df_synthetic_pos)} data Judol sintetik ditambahkan.")
print(f"✅ {len(df_neg_aug)} data Non-Judol (negatif) augmentasi ditambahkan.")

# ===============================================
# 3. FUNGSI PREPROCESSING KONSISTEN
# (Mencakup Normalisasi Samaran, Stopword, dan Stemming)
# ===============================================
def clean_text_prep(text):
    """Fungsi cleaning konsisten dengan Stemming dan Normalisasi Samaran."""
    if not isinstance(text, str):
        text = str(text)

    # 1. Normalisasi Unicode & Konversi ke ASCII
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    
    # 2. Lowercase, Hapus URL/Mention
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+|@\S+", "", text)
    
    # 3. Hapus karakter yang bukan a-z, 0-9, atau spasi
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    
    # --- NORMALISASI SAMARAN AGRESIF ---
    text_parts = text.split()
    processed_parts = []
    i = 0
    while i < len(text_parts):
        current_part = text_parts[i]
        j = i + 1
        merged_word = current_part
        while j < len(text_parts) and len(text_parts[j]) <= 1 and re.match(r'[a-z0-9]', text_parts[j]):
            merged_word += text_parts[j]
            j += 1
        
        if len(merged_word) > len(current_part):
            processed_parts.append(merged_word)
            i = j
        else:
            processed_parts.append(current_part)
            i += 1

    text = " ".join(processed_parts)
    # ----------------------------------------

    # 4. Rapikan spasi dan trim
    text = re.sub(r"\s+", " ", text).strip()
    
    # 5. STOPWORD REMOVAL (Sastrawi)
    text = stopword_remover.remove(text)
    
    # 6. STEMMING (Sastrawi)
    text = stemmer.stem(text)

    # 7. Rapikan spasi lagi
    text = re.sub(r"\s+", " ", text).strip()
    
    return text


# ===============================================
# 4. LOAD & GABUNGKAN DATA
# ===============================================
print("\n--- 3. Memuat Data, Menggabungkan Augmentasi, dan Preprocessing ---")
try:
    df_original = pd.read_csv(FILE_PATH)
    df_original = df_original.dropna(subset=['comment_text', TARGET_COLUMN])
    df_original[TARGET_COLUMN] = df_original[TARGET_COLUMN].astype(int)
    
    df_combined = pd.concat([
        df_original[['comment_text', TARGET_COLUMN]], 
        df_synthetic_pos,
        df_neg_aug
    ], ignore_index=True)

    print("Menerapkan cleaning, stopword removal, dan stemming...")
    df_combined[TEXT_COLUMN] = df_combined['comment_text'].fillna('').progress_apply(clean_text_prep)
    
    df_combined = df_combined[df_combined[TEXT_COLUMN].str.len() > 0]

except FileNotFoundError:
    print(f"❌ ERROR: File '{FILE_PATH}' tidak ditemukan. Pastikan file berada di direktori yang sama.")
    sys.exit()

X = df_combined[TEXT_COLUMN]
Y = df_combined[TARGET_COLUMN]

print(f"\nTotal data akhir setelah semua augmentasi dan cleaning: {len(X)} baris.")
print(f"Jumlah Judol (1) sebelum split: {Y.sum()}")


# ===============================================
# 5. SPLIT DATA (STRATIFIED)
# ===============================================
print("\n--- 4. Split Data (80% Train, 20% Test Stratified) ---")
X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    X, 
    Y, 
    test_size=TEST_SIZE_FINAL, 
    random_state=SEED, 
    stratify=Y 
)

# ===============================================
# 6. FEATURE EXTRACTION (Tokenisasi & Padding)
# ===============================================
print("\n--- 5. Feature Extraction: Tokenisasi dan Padding ---")
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text.astype(str))

TOKENIZER_PATH = 'tokenizer_augmented.pickle'
with open(TOKENIZER_PATH, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"✅ Tokenizer disimpan sebagai '{TOKENIZER_PATH}'")

X_train_seq = tokenizer.texts_to_sequences(X_train_text.astype(str))
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')

X_test_seq = tokenizer.texts_to_sequences(X_test_text.astype(str))
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')


# ===============================================
# 7. TERAPKAN SMOTE (HANYA PADA DATA LATIH)
# ===============================================
print("\n--- 6. Terapkan SMOTE (Oversampling pada Data Latih) ---")
Y_train_np = Y_train.values 
smote = SMOTE(sampling_strategy='minority', random_state=SEED)

X_resampled, Y_resampled = smote.fit_resample(X_train_padded, Y_train_np)

X_train_final = np.array(X_resampled)
Y_train_final = np.array(Y_resampled)

judol_count = Y_train_final.sum()
non_judol_count = len(Y_train_final) - judol_count
print(f"✅ Data Latih Akhir Setelah SMOTE: {len(X_train_final)} baris (Rasio 1:1)")


# ===============================================
# 8. TRAINING DENGAN DATA SMOTE DAN FOCAL LOSS
# ===============================================
# Definisi Focal Loss harus dipertahankan karena digunakan di compile()
def focal_loss(gamma=2., alpha=NEW_ALPHA):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        loss = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
                     - (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss)
    return loss
    
model_aug = Sequential([
    Embedding(MAX_WORDS, EMBED_DIM, input_length=MAX_LEN),
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
    GlobalMaxPool1D(),
    Dense(128, activation="relu"),
    Dropout(0.4),
    Dense(1, activation="sigmoid")
])

# NOTE: Saat meng-compile, kita masih menggunakan custom loss
model_aug.compile(loss=focal_loss(), optimizer=Adam(learning_rate=LR), metrics=["accuracy"])

callbacks = [
    EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
]

print(f"\n--- 7. Memulai Pelatihan Model (Alpha={NEW_ALPHA}) ---")

history = model_aug.fit(
    X_train_final, Y_train_final, 
    validation_split=0.1, 
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1 
)

# PERUBAHAN KRITIS: Hanya panggil save() dengan path .keras dan overwrite=True
model_aug.save(
    NEW_MODEL_AUG_PATH, 
    overwrite=True, 
)
print(f"\nModel baru disimpan sebagai '{NEW_MODEL_AUG_PATH}' ✅")


# ===============================================
# 9. EVALUASI DAN UJI MODEL BARU
# ===============================================

print("\n=============================================")
print("📊 EVALUASI AKHIR MODEL PADA DATA UJI MURNI")
print("=============================================")

Y_test_np = Y_test.values 

# NOTE: Jika Anda perlu memuat model di skrip lain, Anda harus menggunakan custom_objects di load_model.
# Contoh: loaded_model = load_model(NEW_MODEL_AUG_PATH, custom_objects={'loss': focal_loss(alpha=NEW_ALPHA)})

loss, accuracy = model_aug.evaluate(X_test_padded, Y_test_np, verbose=0) 

print(f"Loss pada Data Uji: {loss:.4f}")
print(f"Akurasi pada Data Uji: {accuracy:.4f}")

y_pred_proba = model_aug.predict(X_test_padded, verbose=0)
y_pred_classes = (y_pred_proba > OPTIMAL_THRESHOLD).astype(int)

print(f"\nLaporan Klasifikasi (Threshold={OPTIMAL_THRESHOLD}):")
print(classification_report(Y_test_np, y_pred_classes))

In [ ]:
import re
import unicodedata
import pickle
import numpy as np
import pandas as pd
# PENTING: Pustaka Sastrawi diimpor tetapi fungsi Stemming dan Stopword TIDAK DIGUNAKAN.
# from Sastrawi.Stemmer.StemmerFactory import StemmerFactory 
# from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# PENTING: Anda harus menginstal TensorFlow untuk menjalankan kode ini
try:
    from tensorflow.keras.models import load_model
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    import keras
except ImportError:
    print("---------------------------------------------------------------------------------------")
    print("PERINGATAN: Pustaka 'tensorflow' tidak ditemukan. Pastikan sudah terinstal.")
    print("---------------------------------------------------------------------------------------")

# --- CUSTOM OBJECTS FIX (Diperlukan untuk memuat model dengan custom loss) ---
@keras.saving.register_keras_serializable()
def loss(y_true, y_pred):
    """Fungsi placeholder untuk custom loss."""
    return keras.losses.binary_crossentropy(y_true, y_pred)


# --- KONSTANTA & PERUBAHAN ---
MAX_LEN = 50
# Ambang batas tetap 0.6, tetapi kita akan menambahkan deteksi nama situs yang kuat.
THRESHOLD = 0.6 
MODEL_PATH = 'judol_detection_augmented_smote.keras'
TOKENIZER_PATH = 'tokenizer_augmented.pickle'

# Perbaikan A: Daftar nama situs yang wajib ditandai sebagai JUDOL
HARD_JUDOL_NAMES = [
    'hotjepe55', 'arena923', 'megabet189', 'labag91', 
    'tokowin303', 'gacorwd77', 'mantul404', 'galabet688',
    'slotvip777', 'maxwin888', 'bossku777', 'rajaslot777', 
    'mantapjepe888', 'slotvip4d', 'mantap888', 'pulauwin88', 
    'slotvip111', 'garudahoki', 'sg188', 'cptlv888'
]

# --- FUNGSI PRA-PEMROSESAN YANG DITINGKATKAN (FIXED V2) ---
def clean_text_prep(text):
    """
    Melakukan pra-pemrosesan yang agresif pada samaran/obfuscation, 
    MEMPERTAHANKAN konteks, dan MENANDAI nama-nama situs judi (HARD-JUDOL).
    """
    # 1. Unicode Normalization (Mengatasi Fancy Fonts)
    text = unicodedata.normalize('NFKC', text)

    # 2. Case Folding (Mengubah ke huruf kecil)
    text = text.lower()

    # 3. Membersihkan Samaran & Simbol
    text = re.sub(r'[\@\$\!\#\^\&\*\(\)\-=\+\|\\\[\]\{\}\;\:\'\"\<\,\>\.\/\?]', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', ' ', text) 

    # 4. Konsolidasi spasi
    text = re.sub(r'\s+', ' ', text).strip()
    
    # 5. Perbaikan A: Menandai Nama Situs Keras (HARD JUDOL)
    # Ini harus dilakukan setelah normalisasi dan pembersihan simbol/spasi.
    words = text.split()
    processed_words = []
    
    for word in words:
        # Cek apakah kata tersebut (atau varian dekatnya) adalah nama situs di daftar HARD_JUDOL_NAMES
        is_hard_judol = False
        for site_name in HARD_JUDOL_NAMES:
            # Gunakan regex untuk mencocokkan kata, mengabaikan angka yang dipecah (misal 9 2 3)
            if re.search(site_name, word):
                is_hard_judol = True
                break
        
        if is_hard_judol:
            # Ganti nama situs terdeteksi dengan token 'SITUSJUDOL'
            processed_words.append('SITUSJUDOL')
        else:
            processed_words.append(word)

    # Gabungkan kembali menjadi teks
    text = ' '.join(processed_words)

    # Finalisasi spasi
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# --- FUNGSI UTAMA DETEKSI ---
def predict_judol(texts, model, tokenizer, max_len=MAX_LEN, threshold=THRESHOLD):
    """
    Melakukan pra-pemrosesan, tokenisasi, prediksi model, dan menentukan label.
    """
    df = pd.DataFrame(texts, columns=['Teks Asli'])
    
    # 1. Pra-pemrosesan (Termasuk penandaan nama situs)
    df['Teks Bersih'] = df['Teks Asli'].apply(clean_text_prep)
    
    # 2. Tokenisasi dan Padding
    sequences = tokenizer.texts_to_sequences(df['Teks Bersih'].tolist())
    padded_sequences = pad_sequences(sequences, maxlen=max_len, padding='post', truncating='post')
    
    # 3. Prediksi
    probabilities = model.predict(padded_sequences, verbose=0).flatten()
    df['Prob. Judol'] = probabilities
    
    # 4. Pelabelan
    df['Label'] = np.where(probabilities >= threshold, 'JUDOL', 'NON-JUDOL')
    
    return df

# --- FUNGSI UTAMA EKSEKUSI DEMO ---
def main():
    print("Mengambil model dari '{}' dan Tokenizer dari '{}'...".format(MODEL_PATH, TOKENIZER_PATH))
    
    try:
        # Tentukan objek kustom yang diperlukan untuk memuat model
        custom_objects_map = {'loss': loss}
        
        # Pemuatan Model dan Tokenizer (Membutuhkan file yang ada)
        model = load_model(MODEL_PATH, custom_objects=custom_objects_map)
        with open(TOKENIZER_PATH, 'rb') as handle:
            tokenizer = pickle.load(handle)
        
        print("✅ Model dan Tokenizer berhasil dimuat.")
        
    except FileNotFoundError:
        print("❌ ERROR: File model atau tokenizer tidak ditemukan. Pastikan kedua file ada.")
        return
    except Exception as e:
        print(f"❌ ERROR: Gagal memuat model atau tokenizer. Detail: {e}")
        return

    # Data Uji Coba Batch (Dari prompt pengguna)
    test_texts = [
        "Judi slot gacor hari ini pasti menang deposit 10k di link bio kami", # JUDOL
        "Selamat siang semua, jangan lupa makan siang ya! Ini adalah cuaca yang cerah sekali.", # NON-JUDOL
        "main di situs MPO444 sekarang dan langsung withdraw! dijamin jp", # JUDOL
        "video ini sangat bagus dan mengedukasi sekali tentang sejarah peradaban kuno di mesir.", # NON-JUDOL
        "Ayo join situs bola aman 11betting terpercaya di indonesia dan dapatkan bonus terbesar hari ini.", # JUDOL
        "Kapan episode selanjutnya tayang? Menunggu banget nih. Kami ingin tahu kelanjutannya.", # NON-JUDOL
        "link alternatif 77slot gampang menang nih bosku. klik link di bio.", # JUDOL
        "Selamat ulang tahun untuk teman baikku! Semoga panjang umur dan sehat selalu ya.", # NON-JUDOL
        "Ayo buruan daftar di MPO 7 77 sekarang, dapatkan bonus terbesar!", # JUDOL
        "WD 500rb di GACOR777 mantap langsung cair!", # JUDOL
        "buruan daftar di MANTAPJP888, depo 100k langsung dapat bonus!", # JUDOL
        "main slot pasti untung, depo terus bosku", # JUDOL
        "JP boss 44D mantap", # NON-JUDOL
        "S L 0 T P R 1 M A 8 8 8. Jp 100k wd cepat.", # JUDOL
        "Cepat gabung w1n k3lub, b4rus4n w d s4ld0 5j.", # JUDOL
        "Lagi gacor main di maxwin888 hari ini. Lomba makan.", # JUDOL (Target perbaikan 1)
        "Top banget! Terimakasih 𝗛𝗢𝗧𝗝𝗘𝗣𝗘𝟱𝟱 GOOD JOB!🀄🥰", # JUDOL (Target perbaikan 2)
        "WD 1O jt di BOSSku777. *lgsg cair*!", # JUDOL
        "Depo 100k, WD 20juta di RAJASLOT777. Pasti JP!", # JUDOL
        "Aku di MANTAP888. Gacor terus!", # JUDOL (Target perbaikan 3)
        "buruan daftar di MANTAPJEPE888 promonya gede banget", # JUDOL
        "Paling the best! HOTJEPE55 Memang GOOD JOB banget!", # JUDOL (Target perbaikan 4)
        "main di slotvip4d aja, depo 50 wd 500", # JUDOL
        "gacorwd77 adalah situs paling oke.", # JUDOL (Target perbaikan 5)
        "wd 1000 jp. link ada di bio ya.", # JUDOL
        "jp 2OOm jt sudah di WD. Paling gacor di sini.", # JUDOL
        "S L O T G A C O R 1 2 3 . C O M. Puji Tuhan, rejeki nomplok", # JUDOL
        "asli keren 𝗚𝗔𝗟𝗔𝗕𝗘𝗧𝟲𝟴𝟴 selalu bikin hepyy.", # JUDOL (Target perbaikan 6)
        "tokowin303 4w1 likeReply", # JUDOL (Target perbaikan 7)
        "Saya mau ngasih saran yg baik ya bg kemaren saya main di 𝙈𝘼𝙉𝙏𝙐𝙇𝟰𝟬𝟰 di kasih 22juta", # JUDOL (Target perbaikan 8)
        "𝗟𝗔𝗕𝗔𝟵𝟭 emang the real MVP, selalu bikin kita bahagia dengan kemenangan emang the real MVP, selalu bikin kita bahagia dengan kemenangan", # JUDOL (Target perbaikan 9)
        "slotvip777 depo 50 wd 500", # JUDOL
        "JP boss 44D mantap", # NON-JUDOL
        "💰 WD 5🟢0🟢0k di P U L A U W I N 88! Langsung 🤑 JP! Lomba", # JUDOL
        "S L 0 T G A C 0 R 1 2 3 --- jp 500k wd cepat", # JUDOL
        "𝗠𝗔𝗡𝗧𝗔𝗣𝗝𝗘𝗣𝗘𝟴𝟴🔥🔥 WD 2𝟬𝟬j", # JUDOL
        "W D   1 0 j t .. d i 𝗕𝗢𝗦𝗦𝗞𝘂777 !!", # JUDOL
        "jp besar 💵 WD kecil. Main di s l 0 t v 1 p 1 1 1 sekarang!", # JUDOL
        "p_υ_l_α_υ_w_ι_n88 💰 jp 10J cepat WD!!", # JUDOL
        "garudahoki🔥 mantapjepe888 🏆 jp 100k wd lancar", # JUDOL
        "wd sepuluh juta di slotvip111 🎯 langsung cair", # JUDOL
        "wd cepat + JP 200k main di RAJASLOT777 sekarang", # JUDOL
        "Daft4r di S G 1 ‧ 8‧8‧ bentar lagi ada drop mantap.", # JUDOL
        "mAin di b𝚘𝚜𝚔𝚞7𝟽𝟽 enak bet, wd aman.", # JUDOL
        "bo.s ku. tujuh tujuh tujuh lagi gila2an promo wdee.", # JUDOL
        "m​a​i​n​ ​d​i​ ​b​o​s​k​u​7​7​7​ g​a​s​k​e​n​.", # JUDOL
        "Coba deh di \"QlRTT1QxMDA=\" katanya jam 3 pagi gacor.", # JUDOL
        "Main di cptlv888 lumayan tarikannya. Lomba", # JUDOL (Target perbaikan 10)
        "我 在 博 司 苦 七星 七星 七星 刚 取 款.", # NON-JUDOL
        "b0sku77? gas bet wd ngebut bgt.", # JUDOL
        "b o s k u 7 7 7 ...disusun vertikal.", # JUDOL
        "WD 600 tadi, no cap. That site hits diff fr.", # JUDOL
        "M@!N di B0$KU^777 cuy wd kuzz teroosss.", # JUDOL
        "bo sku 7 7 7 — karena kamera burem jd gini.", # JUDOL
        "Beli skin game baru ah, keren nih!", # NON-JUDOL
        "Ini channel YouTube favoritku, sering update video baru.", # NON-JUDOL
        "Makan malam apa hari ini?", # NON-JUDOL
        "Saya mau nonton film gratis di y0utube, tapi juga WD 5 ribu dari tabungan", # NON-JUDOL
        "WD 50 ribu saja, buat beli kopi.", # NON-JUDOL
        "wah mantap nih, motor baru udah di WD sama pemiliknya", # NON-JUDOL
        "itu tempat bosku yang warna hijau itu loh, yg sering iklan jam 2 pagi.", # JUDOL (Target perbaikan 11)
        "Tadi malem aku pecah telur di tempat sebelah. Tarikan bersih bgt.", # JUDOL
        "Udah, masuk aja ke yang kemarin aku bilang. Yang tiap hari bagi hadiah itu.", # JUDOL
        "Wkwk mantap jp lagi di “platform terpercaya” itu katanya 🤡", # NON-JUDOL
        "bos ku sudah datang tadi pagi bawa hadiah 25k. Gila.", # NON-JUDOL
        "Ingat, keberanian itu modal. Kalau mau hasil besar, tempat itu udah aku rekomendasiin kemarin.", # JUDOL
        "Semoga sukses selalu dan sehat terus ya. Kontennya sangat bermanfaat!", # NON-JUDOL
        "Saya transfer 500 ribu ke rekening teman tadi pagi, semoga lancar.", # NON-JUDOL
        "Main Mobile Legends seru banget, tapi harus top up dulu buat beli diamond.", # NON-JUDOL
        "Saya pesan nasi Padang di warung Bossku tadi, rasanya mantap.", # NON-JUDOL
        "Saya sudah JP (Jelas Puas) sama pelayanan toko ini.", # NON-JUDOL
        "Komentar ini hanya berisi pujian, tidak ada unsur promosi atau judi.", # NON-JUDOL
        "link untuk download materi kuliah ada di deskripsi video ya teman-teman.", # NON-JUDOL
        "Terima kasih atas video tutorial cara WD (Withdrawal) uang dari platform saham.", # NON-JUDOL
        "Dijamin happy udah kaya rumah kedua 𝐀𝐑𝐄𝐍𝐀𝟗𝟐𝟑 disini 👏", # JUDOL (Target perbaikan 12)
        "cuma di tempat lu bang yang paling amanah🙌", # JUDOL
        "𝙈𝙀𝙂𝘼𝘽𝙀𝙏𝟭𝟴𝟵 emang the real MVP selalu bikin kita bahagia dengan kemenangannya", # JUDOL (Target perbaikan 13)
        "Tadi pagi aku WD (Withdraw) uang tunai dari ATM 500 ribu.", # NON-JUDOL
        "WD dana beasiswa sudah cair, total 5 juta! Alhamdulillah.", # NON-JUDOL
        "Jangan lupa WD hasil penjualan dari platform e-commerce itu ya.", # NON-JUDOL
        "Pelayanan toko ini JP (Jelas Puas) banget, barangnya 'Gacor'!", # NON-JUDOL
        "Aku JP (Jaminan Produk) 100% kalau kamu beli di Bossku Official Store.", # NON-JUDOL
        "Wah, si Bossku tadi pagi datang bawa hadiah, mantap JP!", # NON-JUDOL
        "Motor baruku Gacor banget, tarikannya mantap!", # NON-JUDOL
        "Si Bossku (panggilan teman) tadi Depo (Deposit) makanan ke meja.", # NON-JUDOL
        "Restoran ini Gacor tiap hari, ramenya nggak ketulungan.", # NON-JUDOL
        "Aku dapat promo Depo 50k, dapat 50k di Toko X (bukan situs judi).", # NON-JUDOL
        "Gacor banget, main game Mobile Legends dapat JP (Juara Pertama).", # NON-JUDOL
        "Kata Bossku, ini adalah situs terbaik untuk beli akun game (bukan judi).", # NON-JUDOL
        "WD 5 ribu aja buat beli kopi, soalnya dompet udah 'Gacor'." # NON-JUDOL
    ]
    
    print("\n" + "="*80)
    print(f"DEMO PREDIKSI BATCH (Ambangan Batas/Threshold: {THRESHOLD})")
    print(f"Model: {MODEL_PATH}")
    print("Pre-processing: FIXED V2 - Marking Hard Judol Names (e.g., arena923 -> SITUSJUDOL)")
    print("="*80 + "\n")

    # Jalankan Prediksi
    results_df = predict_judol(test_texts, model, tokenizer)

    # Menampilkan Hasil dengan Formatting Rapi
    print("No.  | Prob. Judol  | Label        | Teks Asli")
    print("-----+--------------+--------------+-----------------------------------------------------------------------")
    
    for i, row in results_df.iterrows():
        # Teks Asli dan Teks Bersih
        original_text = row['Teks Asli']
        clean_text = row['Teks Bersih']

        # Baris pertama (dengan Probabilitas dan Label)
        print(f"{i+1:<4} | {row['Prob. Judol']:<12.4f} | {row['Label']:<12} | {original_text:<70}")

        # Tambahkan baris untuk Teks Bersih
        print(f"     |              |              | (Bersih): {clean_text:<70}")
        
        print("-" * 87)
    
    print("\n--- Keterangan ---")
    print(f"Label JUDOL: Promosi Perjudian Online (Probabilitas >= {THRESHOLD})")
    print(f"Label NON-JUDOL: Komentar Aman (Probabilitas < {THRESHOLD})")

if __name__ == '__main__':
    main()

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# --- MODUL WAJIB ---
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import unicodedata
import pickle
import sys
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from tqdm import tqdm
tqdm.pandas()


# ===============================================
# CONFIGURASI UTAMA & INISIALISASI SASTRAWI
# ===============================================
FILE_PATH = "final_production_judol_detection.csv"
MAX_WORDS = 20000
MAX_LEN = 100
EMBED_DIM = 128
BATCH_SIZE = 64
EPOCHS = 25
LR = 1e-3
SEED = 42
# <<< PERUBAHAN KRITIS UNTUK MENINGKATKAN PRECISION
# Alpha diubah dari 0.55 menjadi 0.51 (mendekati 0.50)
# Ini mengurangi bias terhadap kelas Judol, memaksa model lebih konservatif.
NEW_ALPHA = 0.51 
# Threshold diubah dari 0.65 menjadi 0.55.
OPTIMAL_THRESHOLD = 0.55 
# >>>
TEXT_COLUMN = 'cleaned_comment_text'
TARGET_COLUMN = 'target'
TEST_SIZE_FINAL = 0.20
NEW_MODEL_AUG_PATH = "judol_detection_high_precision.keras"

np.random.seed(SEED)
tf.random.set_seed(SEED)

# INISIALISASI SASTRAWI
factory = StopWordRemoverFactory()
stopword_remover = factory.create_stop_word_remover()
factory_stemmer = StemmerFactory()
stemmer = factory_stemmer.create_stemmer()

# ===============================================
# 1. DATA AUGMENTASI: SINTESIS JUDOL (LABEL 1)
# ===============================================
prefixes = ['slot', 'jp', 'wd', 'depo', 'gacor', 'raja', 'bosku', 'mantap']
numbers = [str(i) for i in range(100, 999, 100)] + [str(i) for i in range(11, 99, 11)]
phrases = ['gacor banget di sini!', 'pasti jp di situs ini', 'wd cepat tanpa ribet', 'cair terus bosku']

synthetic_data_pos = []
for i in range(200):
    prefix = np.random.choice(prefixes)
    num = np.random.choice(numbers)
    phrase = np.random.choice(phrases)
    brand_name = prefix + num
    comment = f"{phrase} coba main di {brand_name} aja deh"
    synthetic_data_pos.append({"comment_text": comment, "target": 1})

df_synthetic_pos = pd.DataFrame(synthetic_data_pos)

# ===============================================
# 2. DATA AUGMENTASI: SINTESIS NON-JUDOL (LABEL 0) - PENTING UNTUK PRESISI
# ===============================================
NEGATIVE_AUGMENTATION_DATA = [
    "WD uang beasiswa sudah cair 5 juta alhamdulillah",
    "WD dana dari platform saham tutorialnya jelas",
    "WD gaji dari kantor total 10 juta",
    "WD uang tunai 500 ribu dari ATM",
    "Withdrawal dana P2P lending cair ke rekening",
    "tarik uang 1 juta untuk beli buku kuliah",
    "gacor banget motor tarikan mantap",
    "gacor banget pelayanan toko ini",
    "jp jelas puas sama produknya",
    "motor baru ini gacor sekali",
    "mobil jerman ini tarikannya mantap",
    "jp (jelas puas) sama hasilnya",
    "bosku beli kopi di warung",
    "saya sudah deposit uang di bank",
    "depo uang kuliah semester 3",
    "akun ini sudah top up diamond",
    "kami adalah agen perjalanan resmi",
]

df_neg_aug = pd.DataFrame({'comment_text': NEGATIVE_AUGMENTATION_DATA, 'target': 0})

print(f"✅ {len(df_synthetic_pos)} data Judol sintetik ditambahkan.")
print(f"✅ {len(df_neg_aug)} data Non-Judol (negatif) augmentasi ditambahkan.")

# ===============================================
# 3. FUNGSI PREPROCESSING KONSISTEN
# (Mencakup Normalisasi Samaran, Stopword, dan Stemming)
# ===============================================
def clean_text_prep(text):
    """Fungsi cleaning konsisten dengan Stemming dan Normalisasi Samaran."""
    if not isinstance(text, str):
        text = str(text)

    # 1. Normalisasi Unicode & Konversi ke ASCII
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    
    # 2. Lowercase, Hapus URL/Mention
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+|@\S+", "", text)
    
    # 3. Hapus karakter yang bukan a-z, 0-9, atau spasi
    text = re.sub(r"[^a-z0-9\s]", " ", text)
    
    # --- NORMALISASI SAMARAN AGRESIF ---
    text_parts = text.split()
    processed_parts = []
    i = 0
    while i < len(text_parts):
        current_part = text_parts[i]
        j = i + 1
        merged_word = current_part
        while j < len(text_parts) and len(text_parts[j]) <= 1 and re.match(r'[a-z0-9]', text_parts[j]):
            merged_word += text_parts[j]
            j += 1
        
        if len(merged_word) > len(current_part):
            processed_parts.append(merged_word)
            i = j
        else:
            processed_parts.append(current_part)
            i += 1

    text = " ".join(processed_parts)
    # ----------------------------------------

    # 4. Rapikan spasi dan trim
    text = re.sub(r"\s+", " ", text).strip()
    
    # 5. STOPWORD REMOVAL (Sastrawi)
    text = stopword_remover.remove(text)
    
    # 6. STEMMING (Sastrawi)
    text = stemmer.stem(text)

    # 7. Rapikan spasi lagi
    text = re.sub(r"\s+", " ", text).strip()
    
    return text


# ===============================================
# 4. LOAD & GABUNGKAN DATA
# ===============================================
print("\n--- 3. Memuat Data, Menggabungkan Augmentasi, dan Preprocessing ---")
try:
    df_original = pd.read_csv(FILE_PATH)
    df_original = df_original.dropna(subset=['comment_text', TARGET_COLUMN])
    df_original[TARGET_COLUMN] = df_original[TARGET_COLUMN].astype(int)
    
    df_combined = pd.concat([
        df_original[['comment_text', TARGET_COLUMN]], 
        df_synthetic_pos,
        df_neg_aug
    ], ignore_index=True)

    print("Menerapkan cleaning, stopword removal, dan stemming...")
    df_combined[TEXT_COLUMN] = df_combined['comment_text'].fillna('').progress_apply(clean_text_prep)
    
    df_combined = df_combined[df_combined[TEXT_COLUMN].str.len() > 0]

except FileNotFoundError:
    print(f"❌ ERROR: File '{FILE_PATH}' tidak ditemukan. Pastikan file berada di direktori yang sama.")
    sys.exit()

X = df_combined[TEXT_COLUMN]
Y = df_combined[TARGET_COLUMN]

print(f"\nTotal data akhir setelah semua augmentasi dan cleaning: {len(X)} baris.")
print(f"Jumlah Judol (1) sebelum split: {Y.sum()}")


# ===============================================
# 5. SPLIT DATA (STRATIFIED)
# ===============================================
print("\n--- 4. Split Data (80% Train, 20% Test Stratified) ---")
X_train_text, X_test_text, Y_train, Y_test = train_test_split(
    X, 
    Y, 
    test_size=TEST_SIZE_FINAL, 
    random_state=SEED, 
    stratify=Y 
)

# ===============================================
# 6. FEATURE EXTRACTION (Tokenisasi & Padding)
# ===============================================
print("\n--- 5. Feature Extraction: Tokenisasi dan Padding ---")
tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text.astype(str))

TOKENIZER_PATH = 'tokenizer_high_precision.pickle'
with open(TOKENIZER_PATH, 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(f"✅ Tokenizer disimpan sebagai '{TOKENIZER_PATH}'")

X_train_seq = tokenizer.texts_to_sequences(X_train_text.astype(str))
X_train_padded = pad_sequences(X_train_seq, maxlen=MAX_LEN, padding='post', truncating='post')

X_test_seq = tokenizer.texts_to_sequences(X_test_text.astype(str))
X_test_padded = pad_sequences(X_test_seq, maxlen=MAX_LEN, padding='post', truncating='post')


# ===============================================
# 7. TERAPKAN SMOTE (HANYA PADA DATA LATIH)
# ===============================================
print("\n--- 6. Terapkan SMOTE (Oversampling pada Data Latih) ---")
Y_train_np = Y_train.values 
smote = SMOTE(sampling_strategy='minority', random_state=SEED)

X_resampled, Y_resampled = smote.fit_resample(X_train_padded, Y_train_np)

X_train_final = np.array(X_resampled)
Y_train_final = np.array(Y_resampled)

judol_count = Y_train_final.sum()
non_judol_count = len(Y_train_final) - judol_count
print(f"✅ Data Latih Akhir Setelah SMOTE: {len(X_train_final)} baris (Rasio 1:1)")


# ===============================================
# 8. TRAINING DENGAN DATA SMOTE DAN FOCAL LOSS
# ===============================================
# Definisi Focal Loss harus dipertahankan karena digunakan di compile()
def focal_loss(gamma=2., alpha=NEW_ALPHA):
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        
        # PERHATIAN: Loss dihitung dengan NEW_ALPHA = 0.51
        loss = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
                     - (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss)
    return loss
    
model_aug = Sequential([
    Embedding(MAX_WORDS, EMBED_DIM, input_length=MAX_LEN),
    Bidirectional(LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
    GlobalMaxPool1D(),
    Dense(128, activation="relu"),
    Dropout(0.4),
    Dense(1, activation="sigmoid")
])

# NOTE: Saat meng-compile, kita masih menggunakan custom loss
model_aug.compile(loss=focal_loss(), optimizer=Adam(learning_rate=LR), metrics=["accuracy"])

callbacks = [
    EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, verbose=1),
]

print(f"\n--- 7. Memulai Pelatihan Model (Alpha={NEW_ALPHA} untuk Precision Tinggi) ---")

history = model_aug.fit(
    X_train_final, Y_train_final, 
    validation_split=0.1, 
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1 
)

model_aug.save(
    NEW_MODEL_AUG_PATH, 
    overwrite=True, 
)
print(f"\nModel baru disimpan sebagai '{NEW_MODEL_AUG_PATH}' ✅")


# ===============================================
# 9. EVALUASI DAN UJI MODEL BARU
# ===============================================

print("\n=============================================")
print("📊 EVALUASI AKHIR MODEL PADA DATA UJI MURNI")
print("=============================================")

Y_test_np = Y_test.values 

loss, accuracy = model_aug.evaluate(X_test_padded, Y_test_np, verbose=0) 

print(f"Loss pada Data Uji: {loss:.4f}")
print(f"Akurasi pada Data Uji: {accuracy:.4f}")

y_pred_proba = model_aug.predict(X_test_padded, verbose=0)
y_pred_classes = (y_pred_proba > OPTIMAL_THRESHOLD).astype(int)

print(f"\nLaporan Klasifikasi (Threshold={OPTIMAL_THRESHOLD}):")
print(classification_report(Y_test_np, y_pred_classes))

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
import unicodedata
import pickle
import sys
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# ===============================================
# KONFIGURASI DAN INFERENSI
# ===============================================
MAX_LEN = 100
# Menggunakan Alpha dan Threshold sebelumnya (0.51 dan 0.30)
# Perubahan fokus pada perbaikan fitur (preprocessing)
NEW_ALPHA = 0.51 
OPTIMAL_THRESHOLD = 0.30 
TOKENIZER_PATH = 'tokenizer_high_recall_weighted.pickle'
MODEL_PATH = 'judol_detection_high_recall_weighted.keras'

# TOKEN GENERIK BARU
LINK_TOKEN = '[LINK]'
BRAND_TOKEN = '[BRAND]'

# INISIALISASI SASTRAWI
factory = StopWordRemoverFactory()
stopword_remover = factory.create_stop_word_remover()
factory_stemmer = StemmerFactory()
stemmer = factory_stemmer.create_stemmer()

# ===============================================
# FUNGSI WAJIB UNTUK LOAD MODEL
# (Harus identik dengan fungsi saat menyimpan model)
# ===============================================
def focal_loss(gamma=2., alpha=NEW_ALPHA):
    """Definisi Focal Loss yang digunakan untuk load_model."""
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        loss = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred) \
                     - (1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        return tf.reduce_mean(loss)
    return loss

# ===============================================
# FUNGSI PREPROCESSING KONSISTEN & GENERALISASI BRAND
# ===============================================
def clean_text_prep(text):
    """Fungsi cleaning dengan Stemming, Normalisasi Samaran, dan Masking Brand/Link."""
    if not isinstance(text, str):
        text = str(text)

    # 1. Normalisasi Unicode & Konversi ke ASCII
    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    
    # 2. Lowercase, Hapus Mention
    text = text.lower()
    text = re.sub(r"@\S+", "", text)
    
    # --- MASKING ENTITAS (HARUS SEBELUM HAPUS TANDA BACA/SPASI) ---
    
    # 2.1 MASKING URL/LINK: Ganti semua pola link dengan token generik [LINK]
    text = re.sub(r"http\S+|www\S+|https\S+", LINK_TOKEN, text)
    
    # 3. Hapus karakter yang bukan a-z, 0-9, atau spasi (biarkan titik untuk samaran)
    text = re.sub(r"[^a-z0-9\s.]", " ", text) 
    
    # 4. NORMALISASI SAMARAN AGRESIF (s.l.o.t. -> slot, j.p -> jp)
    # Hilangkan spasi di sekitar titik, lalu hilangkan titik
    text = re.sub(r'\s*\.\s*', '.', text)
    text = text.replace('.', '')
    # Rapikan spasi
    text = re.sub(r"\s+", " ", text).strip()
    
    # 5. NORMALISASI SPASI PENDEK (Menggabungkan h t t p s, j p, w d)
    text_parts = text.split()
    processed_parts = []
    i = 0
    while i < len(text_parts):
        current_part = text_parts[i]
        j = i + 1
        merged_word = current_part
        
        # Merge huruf tunggal yang terpisah (misalnya j p -> jp)
        while j < len(text_parts) and len(text_parts[j]) <= 1 and re.match(r'[a-z0-9]', text_parts[j]):
            merged_word += text_parts[j]
            j += 1
        
        if len(merged_word) > len(current_part):
             processed_parts.append(merged_word)
             i = j
        else:
             processed_parts.append(current_part)
             i += 1

    text = " ".join(processed_parts)
    text = re.sub(r"\s+", " ", text).strip()
    
    # 6. MASKING BRAND/NAMA SITUS: Ganti kata yang mengandung 3 atau lebih digit/angka berulang
    def mask_brand(match):
        word = match.group(0)
        # Jika kata mengandung token LINK, jangan diubah
        if LINK_TOKEN in word:
            return word
        # Pola umum brand Judol (angka berulang, angka di tengah)
        if re.search(r'\d{3,}|[a-z]{2,}\d{2,}[a-z]*', word) and word not in ['hari', 'situs']:
             return BRAND_TOKEN
        return word

    text = " ".join([mask_brand(re.match(r'.*', word)) for word in text.split()])
    text = text.replace(f" {LINK_TOKEN}", LINK_TOKEN).replace(f"{LINK_TOKEN} ", LINK_TOKEN) # Clean up spaces around [LINK]
    
    # 7. STOPWORD REMOVAL (Sastrawi)
    text = stopword_remover.remove(text)
    
    # 8. STEMMING (Sastrawi)
    text = stemmer.stem(text)

    # 9. Rapikan spasi lagi
    text = re.sub(r"\s+", " ", text).strip()
    
    return text

# ===============================================
# FUNGSI UTAMA PENGUJIAN
# ===============================================

def predict_judol(text, model, tokenizer, idx):
    """Memproses teks dan mengklasifikasikannya."""
    print("=" * 60)
    print(f"[{idx+1}] TESTING | Teks Asli: '{text}'")
    
    # 1. Preprocessing
    cleaned_text = clean_text_prep(text)
    print(f"-> Teks Bersih: '{cleaned_text}'")
    
    if not cleaned_text:
        print("-> HASIL: Teks kosong setelah pembersihan.")
        print("=" * 60)
        return

    # 2. Tokenisasi & Padding
    seq = tokenizer.texts_to_sequences([cleaned_text])
    padded = pad_sequences(seq, maxlen=MAX_LEN, padding='post', truncating='post')
    
    # 3. Prediksi
    probability = model.predict(padded, verbose=0)[0][0]
    
    # 4. Klasifikasi
    prediction = 1 if probability > OPTIMAL_THRESHOLD else 0
    
    # 5. Output
    result_map = {1: "🚨 BERPOTENSI JUDOL (TARGET 1)", 0: "✅ BUKAN JUDOL (TARGET 0)"}
    
    print(f"-> Probabilitas Judol: {probability:.4f}")
    print(f"-> Threshold Prediksi: {OPTIMAL_THRESHOLD}")
    print(f"-> HASIL: {result_map[prediction]}")
    print("=" * 60)


# ===============================================
# LOAD ASET
# ===============================================
try:
    print(f"⏳ Memuat Tokenizer dari '{TOKENIZER_PATH}'...")
    with open(TOKENIZER_PATH, 'rb') as handle:
        tokenizer = pickle.load(handle)
    print("✅ Tokenizer berhasil dimuat.")
    
    print(f"⏳ Memuat Model Keras dari '{MODEL_PATH}'...")
    # Penting: Menyediakan custom_objects untuk fungsi Focal Loss
    model = load_model(
        MODEL_PATH, 
        custom_objects={'loss': focal_loss(alpha=NEW_ALPHA)}
    )
    print("✅ Model berhasil dimuat. Siap Uji Array.")
    print(f"Menggunakan Threshold Prediksi: {OPTIMAL_THRESHOLD}")
    print("\n")

except FileNotFoundError:
    print("❌ ERROR: Pastikan file 'tokenizer_high_precision.pickle' dan 'judol_detection_high_precision.keras' ada di direktori yang sama.")
    sys.exit()


# ===============================================
# ARRAY TEST CASE (Mencakup skenario Brand-Only dan Samaran)
# ===============================================
TEST_CASES = [
    "Mantap bos, JP 100jt di situs gacor X Y Z langsung WD! Daftar sekarang.", # Judol Jelas
    "JP lagi bosku, wd langsung cair di situs gacor SENSITIF999.", # Judol Jelas (Brand-heavy)
    "s.l.o.t. g.a.c.o.r. jp hari ini 888", # Judol Samaran (Brand di akhir)
    "WD uang gaji dari kantor sudah masuk rekening, alhamdulillah.", # Non-Judol (Kata kunci umum)
    "Saya mau withdraw uang dari tabungan, apakah sudah bisa?", # Non-Judol (Kata kunci umum)
    "Main game online itu menyenangkan, tapi jangan sampai lupa waktu.", # Non-Judol
    "link ALTERNATIF DEPOSIT 10.000 GAMPANG JP: h t t p s://link.gacor.com/promo", # Judol dengan Link Samaran
    "Mabar yuk, main game di server Asia!", # Non-Judol
    "G.A.C.O.R 777 HARI INI PASTI CUAN", # Judol Samaran dan Brand
    "Hanya di VIPWIN888, cuan setiap hari!", # Brand-Only (Mengecek Generalisasi)
    "Situs paling jitu cuma di raja777, gak pake lama!", # Brand-Only
     "Judi slot gacor hari ini pasti menang deposit 10k di link bio kami", # JUDOL
        "Selamat siang semua, jangan lupa makan siang ya! Ini adalah cuaca yang cerah sekali.", # NON-JUDOL
        "main di situs MPO444 sekarang dan langsung withdraw! dijamin jp", # JUDOL
        "video ini sangat bagus dan mengedukasi sekali tentang sejarah peradaban kuno di mesir.", # NON-JUDOL
        "Ayo join situs bola aman 11betting terpercaya di indonesia dan dapatkan bonus terbesar hari ini.", # JUDOL
        "Kapan episode selanjutnya tayang? Menunggu banget nih. Kami ingin tahu kelanjutannya.", # NON-JUDOL
        "link alternatif 77slot gampang menang nih bosku. klik link di bio.", # JUDOL
        "Selamat ulang tahun untuk teman baikku! Semoga panjang umur dan sehat selalu ya.", # NON-JUDOL
        "Ayo buruan daftar di MPO 7 77 sekarang, dapatkan bonus terbesar!", # JUDOL
        "WD 500rb di GACOR777 mantap langsung cair!", # JUDOL
        "buruan daftar di MANTAPJP888, depo 100k langsung dapat bonus!", # JUDOL
        "main slot pasti untung, depo terus bosku", # JUDOL
        "JP boss 44D mantap", # NON-JUDOL
        "S L 0 T P R 1 M A 8 8 8. Jp 100k wd cepat.", # JUDOL
        "Cepat gabung w1n k3lub, b4rus4n w d s4ld0 5j.", # JUDOL
        "Lagi gacor main di maxwin888 hari ini. Lomba makan.", # JUDOL (Target perbaikan 1)
        "Top banget! Terimakasih 𝗛𝗢𝗧𝗝𝗘𝗣𝗘𝟱𝟱 GOOD JOB!🀄🥰", # JUDOL (Target perbaikan 2)
        "WD 1O jt di BOSSku777. *lgsg cair*!", # JUDOL
        "Depo 100k, WD 20juta di RAJASLOT777. Pasti JP!", # JUDOL
        "Aku di MANTAP888. Gacor terus!", # JUDOL (Target perbaikan 3)
        "buruan daftar di MANTAPJEPE888 promonya gede banget", # JUDOL
        "Paling the best! HOTJEPE55 Memang GOOD JOB banget!", # JUDOL (Target perbaikan 4)
        "main di slotvip4d aja, depo 50 wd 500", # JUDOL
        "gacorwd77 adalah situs paling oke.", # JUDOL (Target perbaikan 5)
        "wd 1000 jp. link ada di bio ya.", # JUDOL
        "jp 2OOm jt sudah di WD. Paling gacor di sini.", # JUDOL
        "S L O T G A C O R 1 2 3 . C O M. Puji Tuhan, rejeki nomplok", # JUDOL
        "asli keren 𝗚𝗔𝗟𝗔𝗕𝗘𝗧𝟲𝟴𝟴 selalu bikin hepyy.", # JUDOL (Target perbaikan 6)
        "tokowin303 4w1 likeReply", # JUDOL (Target perbaikan 7)
        "Saya mau ngasih saran yg baik ya bg kemaren saya main di 𝙈𝘼𝙉𝙏𝙐𝙇𝟰𝟬𝟰 di kasih 22juta", # JUDOL (Target perbaikan 8)
        "𝗟𝗔𝗕𝗔𝟵𝟭 emang the real MVP, selalu bikin kita bahagia dengan kemenangan emang the real MVP, selalu bikin kita bahagia dengan kemenangan", # JUDOL (Target perbaikan 9)
        "slotvip777 depo 50 wd 500", # JUDOL
        "JP boss 44D mantap", # NON-JUDOL
        "💰 WD 5🟢0🟢0k di P U L A U W I N 88! Langsung 🤑 JP! Lomba", # JUDOL
        "S L 0 T G A C 0 R 1 2 3 --- jp 500k wd cepat", # JUDOL
        "𝗠𝗔𝗡𝗧𝗔𝗣𝗝𝗘𝗣𝗘𝟴𝟴🔥🔥 WD 2𝟬𝟬j", # JUDOL
        "W D   1 0 j t .. d i 𝗕𝗢𝗦𝗦𝗞𝘂777 !!", # JUDOL
        "jp besar 💵 WD kecil. Main di s l 0 t v 1 p 1 1 1 sekarang!", # JUDOL
        "p_υ_l_α_υ_w_ι_n88 💰 jp 10J cepat WD!!", # JUDOL
        "garudahoki🔥 mantapjepe888 🏆 jp 100k wd lancar", # JUDOL
        "wd sepuluh juta di slotvip111 🎯 langsung cair", # JUDOL
        "wd cepat + JP 200k main di RAJASLOT777 sekarang", # JUDOL
        "Daft4r di S G 1 ‧ 8‧8‧ bentar lagi ada drop mantap.", # JUDOL
        "mAin di b𝚘𝚜𝚔𝚞7𝟽𝟽 enak bet, wd aman.", # JUDOL
        "bo.s ku. tujuh tujuh tujuh lagi gila2an promo wdee.", # JUDOL
        "m​a​i​n​ ​d​i​ ​b​o​s​k​u​7​7​7​ g​a​s​k​e​n​.", # JUDOL
        "Coba deh di \"QlRTT1QxMDA=\" katanya jam 3 pagi gacor.", # JUDOL
        "Main di cptlv888 lumayan tarikannya. Lomba", # JUDOL (Target perbaikan 10)
        "我 在 博 司 苦 七星 七星 七星 刚 取 款.", # NON-JUDOL
        "b0sku77? gas bet wd ngebut bgt.", # JUDOL
        "b o s k u 7 7 7 ...disusun vertikal.", # JUDOL
        "WD 600 tadi, no cap. That site hits diff fr.", # JUDOL
        "M@!N di B0$KU^777 cuy wd kuzz teroosss.", # JUDOL
        "bo sku 7 7 7 — karena kamera burem jd gini.", # JUDOL
        "Beli skin game baru ah, keren nih!", # NON-JUDOL
        "Ini channel YouTube favoritku, sering update video baru.", # NON-JUDOL
        "Makan malam apa hari ini?", # NON-JUDOL
        "Saya mau nonton film gratis di y0utube, tapi juga WD 5 ribu dari tabungan", # NON-JUDOL
        "WD 50 ribu saja, buat beli kopi.", # NON-JUDOL
        "wah mantap nih, motor baru udah di WD sama pemiliknya", # NON-JUDOL
        "itu tempat bosku yang warna hijau itu loh, yg sering iklan jam 2 pagi.", # JUDOL (Target perbaikan 11)
        "Tadi malem aku pecah telur di tempat sebelah. Tarikan bersih bgt.", # JUDOL
        "Udah, masuk aja ke yang kemarin aku bilang. Yang tiap hari bagi hadiah itu.", # JUDOL
        "Wkwk mantap jp lagi di “platform terpercaya” itu katanya 🤡", # NON-JUDOL
        "bos ku sudah datang tadi pagi bawa hadiah 25k. Gila.", # NON-JUDOL
        "Ingat, keberanian itu modal. Kalau mau hasil besar, tempat itu udah aku rekomendasiin kemarin.", # JUDOL
        "Semoga sukses selalu dan sehat terus ya. Kontennya sangat bermanfaat!", # NON-JUDOL
        "Saya transfer 500 ribu ke rekening teman tadi pagi, semoga lancar.", # NON-JUDOL
        "Main Mobile Legends seru banget, tapi harus top up dulu buat beli diamond.", # NON-JUDOL
        "Saya pesan nasi Padang di warung Bossku tadi, rasanya mantap.", # NON-JUDOL
        "Saya sudah JP (Jelas Puas) sama pelayanan toko ini.", # NON-JUDOL
        "Komentar ini hanya berisi pujian, tidak ada unsur promosi atau judi.", # NON-JUDOL
        "link untuk download materi kuliah ada di deskripsi video ya teman-teman.", # NON-JUDOL
        "Terima kasih atas video tutorial cara WD (Withdrawal) uang dari platform saham.", # NON-JUDOL
        "Dijamin happy udah kaya rumah kedua 𝐀𝐑𝐄𝐍𝐀𝟗𝟐𝟑 disini 👏", # JUDOL (Target perbaikan 12)
        "cuma di tempat lu bang yang paling amanah🙌", # JUDOL
        "𝙈𝙀𝙂𝘼𝘽𝙀𝙏𝟭𝟴𝟵 emang the real MVP selalu bikin kita bahagia dengan kemenangannya", # JUDOL (Target perbaikan 13)
        "Tadi pagi aku WD (Withdraw) uang tunai dari ATM 500 ribu.", # NON-JUDOL
        "WD dana beasiswa sudah cair, total 5 juta! Alhamdulillah.", # NON-JUDOL
        "Jangan lupa WD hasil penjualan dari platform e-commerce itu ya.", # NON-JUDOL
        "Pelayanan toko ini JP (Jelas Puas) banget, barangnya 'Gacor'!", # NON-JUDOL
        "Aku JP (Jaminan Produk) 100% kalau kamu beli di Bossku Official Store.", # NON-JUDOL
        "Wah, si Bossku tadi pagi datang bawa hadiah, mantap JP!", # NON-JUDOL
        "Motor baruku Gacor banget, tarikannya mantap!", # NON-JUDOL
        "Si Bossku (panggilan teman) tadi Depo (Deposit) makanan ke meja.", # NON-JUDOL
        "Restoran ini Gacor tiap hari, ramenya nggak ketulungan.", # NON-JUDOL
        "Aku dapat promo Depo 50k, dapat 50k di Toko X (bukan situs judi).", # NON-JUDOL
        "Gacor banget, main game Mobile Legends dapat JP (Juara Pertama).", # NON-JUDOL
        "Kata Bossku, ini adalah situs terbaik untuk beli akun game (bukan judi).", # NON-JUDOL
        "WD 5 ribu aja buat beli kopi, soalnya dompet udah 'Gacor'." # NON-JUDOL
]

def array_test_loop():
    print("--- MEMULAI PENGUJIAN DENGAN ARRAY INPUT ---")
    for i, text in enumerate(TEST_CASES):
        predict_judol(text, model, tokenizer, i)
    print("--- PENGUJIAN ARRAY SELESAI ---")

if __name__ == "__main__":
    array_test_loop()

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
import re
import unicodedata
import pickle
import sys
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from tqdm import tqdm # Import tqdm untuk progress bar

# Aktifkan progress bar untuk operasi Pandas/Series
tqdm.pandas() 

# ===============================================
# KONFIGURASI DAN HYPERPARAMETER
# ===============================================
DATA_FILE = 'production_judol_detection_old.csv'
MODEL_PATH = 'judol_detection_high_recall_weighted.keras' 
TOKENIZER_PATH = 'tokenizer_high_recall_weighted.pickle'
RANDOM_STATE = 42

VOCAB_SIZE = 5000
MAX_LEN = 100
EMBEDDING_DIM = 128
LSTM_UNITS = 64
EPOCHS = 10
BATCH_SIZE = 32

# Alpha baru untuk Focal Loss: 
NEW_ALPHA = 0.35 

# TOKEN GENERIK UNTUK MASKING
LINK_TOKEN = '[LINK]'
BRAND_TOKEN = '[BRAND]'

# INISIALISASI SASTRAWI
try:
    factory = StopWordRemoverFactory()
    stopword_remover = factory.create_stop_word_remover()
    factory_stemmer = StemmerFactory()
    stemmer = factory_stemmer.create_stemmer()
except ImportError:
    print("⚠️ Peringatan: Library Sastrawi mungkin tidak terinstal. Preprocessing akan dibatasi.")
    stopword_remover = lambda x: x
    stemmer = lambda x: x

# ===============================================
# FUNGSI PREPROCESSING KONSISTEN
# ===============================================

def clean_text_prep(text):
    """Fungsi cleaning dengan Stemming, Normalisasi Samaran, dan Masking Brand/Link."""
    if not isinstance(text, str):
        text = str(text)

    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    text = text.lower()
    text = re.sub(r"@\S+", "", text)
    
    # MASKING URL/LINK
    text = re.sub(r"http\S+|www\S+|https\S+", LINK_TOKEN, text)
    
    text = re.sub(r"[^a-z0-9\s.]", " ", text) 
    text = re.sub(r'\s*\.\s*', '.', text)
    text = text.replace('.', '')
    text = re.sub(r"\s+", " ", text).strip()
    
    # NORMALISASI SPASI PENDEK
    text_parts = text.split()
    processed_parts = []
    i = 0
    while i < len(text_parts):
        current_part = text_parts[i]
        j = i + 1
        merged_word = current_part
        
        while j < len(text_parts) and len(text_parts[j]) <= 1 and re.match(r'[a-z0-9]', text_parts[j]):
            merged_word += text_parts[j]
            j += 1
        
        if len(merged_word) > len(current_part):
             processed_parts.append(merged_word)
             i = j
        else:
             processed_parts.append(current_part)
             i += 1

    text = " ".join(processed_parts)
    text = re.sub(r"\s+", " ", text).strip()
    
    # MASKING BRAND/NAMA SITUS
    def mask_brand(word):
        if LINK_TOKEN in word:
            return word
        if re.search(r'\d{3,}|[a-z]{2,}\d{2,}[a-z]*', word) and word not in ['hari', 'situs', 'tahun', 'minggu']:
             return BRAND_TOKEN
        return word

    text = " ".join([mask_brand(word) for word in text.split()])
    text = re.sub(r"\s+", " ", text).strip()

    # STOPWORD REMOVAL & STEMMING
    if stopword_remover is not None:
        text = stopword_remover.remove(text)
    
    if stemmer is not None:
        text = stemmer.stem(text)

    text = re.sub(r"\s+", " ", text).strip()
    return text

# ===============================================
# FUNGSI LOSS BARU (FOCAL LOSS)
# ===============================================

def focal_loss(gamma=2., alpha=NEW_ALPHA):
    """Focal Loss dengan alpha rendah untuk Recall Tinggi."""
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        
        loss_pos = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred)
        loss_neg = -(1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        
        return tf.reduce_mean(loss_pos + loss_neg)
    return loss


# ===============================================
# PELATIHAN MODEL UTAMA DENGAN CLASS WEIGHTS
# ===============================================

def train_judol_model():
    print("--- MEMULAI PELATIHAN MODEL REKAL TINGGI DENGAN BOBOT KELAS ---")
    
    try:
        df = pd.read_csv(DATA_FILE)
    except FileNotFoundError:
        print(f"❌ ERROR: File data '{DATA_FILE}' tidak ditemukan.")
        return

    REQUIRED_COLUMNS = ['combined_text', 'target']
    if not all(col in df.columns for col in REQUIRED_COLUMNS):
        print(f"❌ ERROR: Data tidak memiliki kolom yang wajib: {REQUIRED_COLUMNS}")
        return

    # 1. Preprocessing Data
    print("⏳ Menerapkan Preprocessing ke Data...")
    # Menggunakan .progress_apply() dari tqdm
    df['text_cleaned'] = df['combined_text'].astype(str).progress_apply(clean_text_prep)
    
    df_clean = df[df['text_cleaned'].str.len() > 0].copy()
    df_clean['target'] = df_clean['target'].fillna(0).astype(int)

    texts = df_clean['text_cleaned'].tolist()
    labels = df_clean['target'].values
    
    print(f"Total Sampel yang Digunakan: {len(df_clean)}")
    print("\n--- Distribusi Label Data Lengkap ---")
    label_counts = df_clean['target'].value_counts()
    print(label_counts)

    # 2. Split Data (Stratified 80% Train, 20% Test)
    X_train, X_test, y_train, y_test = train_test_split(
        texts, labels, test_size=0.20, random_state=RANDOM_STATE, stratify=labels
    )
    
    # 3. MENGHITUNG BOBOT KELAS (CLASS WEIGHTS)
    print("\n⏳ Menghitung Bobot Kelas untuk mengatasi ketidakseimbangan...")
    
    weights = class_weight.compute_class_weight(
        class_weight='balanced',
        classes=np.unique(y_train),
        y=y_train
    )
    class_weight_dict = {i: weights[i] for i in range(len(weights))}
    
    print(f"Bobot Kelas yang Dihitung (Class Weights):")
    print(f"Kelas 0 (Non-Judol/Mayoritas): {class_weight_dict.get(0):.3f}")
    print(f"Kelas 1 (Judol/Minoritas): {class_weight_dict.get(1):.3f}")
    
    # 4. Tokenisasi
    tokenizer = Tokenizer(num_words=VOCAB_SIZE, oov_token="<unk>")
    tokenizer.fit_on_texts(X_train)
    
    if LINK_TOKEN not in tokenizer.word_index:
        tokenizer.word_index[LINK_TOKEN] = len(tokenizer.word_index) + 1
    if BRAND_TOKEN not in tokenizer.word_index:
        tokenizer.word_index[BRAND_TOKEN] = len(tokenizer.word_index) + 1
    
    train_sequences = tokenizer.texts_to_sequences(X_train)
    padded_train_sequences = pad_sequences(train_sequences, maxlen=MAX_LEN, padding='post', truncating='post')

    # 5. Membangun Model LSTM
    print("\n⏳ Membangun Model LSTM...")
    model = Sequential([
        Embedding(len(tokenizer.word_index) + 1, EMBEDDING_DIM, input_length=MAX_LEN),
        LSTM(LSTM_UNITS, return_sequences=True),
        LSTM(LSTM_UNITS),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid') 
    ])

    # 6. Kompilasi dengan Focal Loss
    print(f"⏳ Mengkompilasi model dengan Focal Loss (alpha={NEW_ALPHA})...")
    model.compile(
        optimizer='adam', 
        loss=focal_loss(gamma=2.0, alpha=NEW_ALPHA), 
        metrics=['accuracy', tf.keras.metrics.Recall(name='recall'), tf.keras.metrics.Precision(name='precision')]
    )

    # 7. Pelatihan dengan Class Weights
    print("⏳ Melatih model dengan Class Weights yang diterapkan...")
    model.fit(
        padded_train_sequences, 
        y_train, 
        epochs=EPOCHS, 
        batch_size=BATCH_SIZE, 
        validation_split=0.1, 
        class_weight=class_weight_dict, 
        verbose=1
    )

    # 8. Menyimpan Model dan Tokenizer
    model.save(MODEL_PATH)
    with open(TOKENIZER_PATH, 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print("\n✅ PELATIHAN SELESAI!")
    print(f"Model Rekal Tinggi tersimpan di: {MODEL_PATH}")
    print(f"Tokenizer tersimpan di: {TOKENIZER_PATH}")
    
    df_test = pd.DataFrame({'combined_text': X_test, 'target': y_test})
    df_test.to_csv('testing_data_split_weighted.csv', index=False)
    print(f"Data testing (20% dari dataset) disimpan di 'testing_data_split_weighted.csv' untuk evaluasi.")
    print("\n**LANGKAH PENGUJIAN:** Jalankan 'judol_testing_high_recall_weighted.py' dengan threshold rendah (0.20) untuk mendapatkan Rekal maksimum!")
    print("--------------------------------------------------")

if __name__ == "__main__":
    train_judol_model()

In [ ]:
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import recall_score, precision_score, f1_score, confusion_matrix, classification_report
import pickle
import unicodedata
from tqdm.auto import tqdm

# Import Sastrawi secara opsional jika diperlukan untuk fungsi preprocessing
try:
    from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
    from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
    factory_stemmer = StemmerFactory()
    stemmer = factory_stemmer.create_stemmer()
    factory_stopword = StopWordRemoverFactory()
    stopword_remover = factory_stopword.create_stop_word_remover()
except ImportError:
    # Fallback jika Sastrawi tidak tersedia, Preprocessing akan terpengaruh
    stemmer = None
    stopword_remover = None
    print("⚠️ Peringatan: Sastrawi tidak ditemukan. Stemming/Stopword Removal dinonaktifkan.")

tqdm.pandas()

# --- 1. KONFIGURASI DAN HYPERPARAMETERS KONSISTEN ---
# HARUS SAMA PERSIS dengan skrip pelatihan Anda
MAX_LEN = 100
NEW_ALPHA = 0.35 

# Ambang Batas Klasifikasi (Threshold) untuk Recall Tinggi
# Nilai rendah ini adalah STRATEGI untuk High Recall.
OPTIMAL_THRESHOLD = 0.20 

# Jalur File
MODEL_PATH = 'judol_detection_high_recall_weighted.keras'
TOKENIZER_PATH = 'tokenizer_high_recall_weighted.pickle'
DATA_FILE = 'testing_data_split_weighted.csv' # Data testing yang dihasilkan skrip pelatihan

# TOKEN GENERIK KONSISTEN
LINK_TOKEN = '[LINK]'
BRAND_TOKEN = '[BRAND]'

# --- 2. FUNGSI PREPROCESSING KONSISTEN (Duplikat dari Skrip Pelatihan) ---
def clean_text_prep(text):
    """Fungsi cleaning dengan Stemming, Normalisasi Samaran, dan Masking Brand/Link."""
    if not isinstance(text, str):
        text = str(text)

    text = unicodedata.normalize('NFKD', text)
    text = text.encode('ascii', 'ignore').decode('utf-8')
    text = text.lower()
    text = re.sub(r"@\S+", "", text)
    
    # MASKING URL/LINK
    text = re.sub(r"http\S+|www\S+|https\S+", LINK_TOKEN, text)
    
    text = re.sub(r"[^a-z0-9\s.]", " ", text) 
    text = re.sub(r'\s*\.\s*', '.', text)
    text = text.replace('.', '')
    text = re.sub(r"\s+", " ", text).strip()
    
    # NORMALISASI SPASI PENDEK
    text_parts = text.split()
    processed_parts = []
    i = 0
    while i < len(text_parts):
        current_part = text_parts[i]
        j = i + 1
        merged_word = current_part
        
        while j < len(text_parts) and len(text_parts[j]) <= 1 and re.match(r'[a-z0-9]', text_parts[j]):
            merged_word += text_parts[j]
            j += 1
        
        if len(merged_word) > len(current_part):
              processed_parts.append(merged_word)
              i = j
        else:
              processed_parts.append(current_part)
              i += 1

    text = " ".join(processed_parts)
    text = re.sub(r"\s+", " ", text).strip()
    
    # MASKING BRAND/NAMA SITUS
    def mask_brand(word):
        if LINK_TOKEN in word:
            return word
        # Kondisi: kombinasi 3+ digit ATAU (2+ huruf diikuti 2+ digit diikuti 0/lebih huruf)
        # dan BUKAN kata-kata pengecualian
        if re.search(r'\d{3,}|[a-z]{2,}\d{2,}[a-z]*', word) and word not in ['hari', 'situs', 'tahun', 'minggu']:
             return BRAND_TOKEN
        return word

    text = " ".join([mask_brand(word) for word in text.split()])
    text = re.sub(r"\s+", " ", text).strip()

    # STOPWORD REMOVAL & STEMMING (gunakan instance Sastrawi yang dimuat di awal)
    if stopword_remover is not None:
        text = stopword_remover.remove(text)
    
    if stemmer is not None:
        text = stemmer.stem(text)

    text = re.sub(r"\s+", " ", text).strip()
    return text

# --- 3. FUNGSI LOSS KONSISTEN (Wajib untuk Pemuatan Model) ---
def focal_loss(gamma=2., alpha=NEW_ALPHA):
    """Focal Loss dengan alpha rendah untuk Recall Tinggi."""
    def loss(y_true, y_pred):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(y_pred, tf.float32)
        y_pred = tf.clip_by_value(y_pred, tf.keras.backend.epsilon(), 1 - tf.keras.backend.epsilon())
        
        loss_pos = -alpha * y_true * tf.pow(1 - y_pred, gamma) * tf.math.log(y_pred)
        loss_neg = -(1 - alpha) * (1 - y_true) * tf.pow(y_pred, gamma) * tf.math.log(1 - y_pred)
        
        return tf.reduce_mean(loss_pos + loss_neg)
    return loss


# --- 4. EKSEKUSI UTAMA PENGUJIAN ---
def test_judol_model():
    print("--- MEMULAI PENGUJIAN MODEL REKAL TINGGI DENGAN THRESHOLD RENDAH ---")
    
    # Pemuatan Tokenizer
    try:
        with open(TOKENIZER_PATH, 'rb') as handle:
            tokenizer = pickle.load(handle)
        print(f"✅ Tokenizer berhasil dimuat dari: {TOKENIZER_PATH}")
    except FileNotFoundError:
        print(f"❌ ERROR: Tokenizer '{TOKENIZER_PATH}' tidak ditemukan. Jalankan pelatihan terlebih dahulu.")
        return
    
    # Pemuatan Model dan Dependencies
    try:
        custom_objects = {'loss': focal_loss(gamma=2., alpha=NEW_ALPHA)}
        model = load_model(MODEL_PATH, custom_objects=custom_objects)
        print(f"✅ Model berhasil dimuat dari: {MODEL_PATH}")
    except FileNotFoundError:
        print(f"❌ ERROR: Model '{MODEL_PATH}' tidak ditemukan. Jalankan pelatihan terlebih dahulu.")
        return
    except Exception as e:
        print(f"❌ ERROR saat memuat model: Pastikan fungsi 'focal_loss' sama persis.")
        print(f"Detail Error: {e}")
        return

    # Pemuatan Data Uji (Split)
    try:
        df_test = pd.read_csv(DATA_FILE)
        X_test_raw = df_test['combined_text'].fillna('')
        y_test = df_test['target'].values
        print(f"✅ Data uji ({len(df_test)} sampel) berhasil dimuat dari: {DATA_FILE}")
    except FileNotFoundError:
        print(f"❌ ERROR: Data uji '{DATA_FILE}' tidak ditemukan. Ini dibuat setelah pelatihan.")
        return

    # --- PREPROCESSING DATA UJI ---
    print("\n[STEP 1/3] Menerapkan Preprocessing ke Data Uji...")
    X_test_processed = X_test_raw.progress_apply(clean_text_prep)
    
    # --- TOKENISASI DAN PADDING ---
    print("[STEP 2/3] Menerapkan Tokenisasi dan Padding...")
    
    # Konversi teks menjadi sequence menggunakan tokenizer yang sudah dilatih
    X_test_sequences = tokenizer.texts_to_sequences(X_test_processed)
    
    # Padding
    X_test_padded = pad_sequences(
        X_test_sequences, 
        maxlen=MAX_LEN, 
        padding='post', 
        truncating='post'
    )

    # --- PREDIKSI MODEL ---
    print("\n[STEP 3/3] Melakukan Prediksi dengan Model...")
    y_pred_proba = model.predict(X_test_padded)
    
    # Ekstrak probabilitas untuk kelas 1 (Judol)
    y_pred_proba = y_pred_proba.flatten() 

    # --- EVALUASI DENGAN THRESHOLD TINGGI RECALL ---
    print(f"\n--- EVALUASI KINERJA MODEL (Threshold: {OPTIMAL_THRESHOLD}) ---")
    print("Tujuan: Rekal Tinggi (Minimalisir False Negatives)")
    
    # Klasifikasi menggunakan threshold rendah
    y_pred_classified = (y_pred_proba >= OPTIMAL_THRESHOLD).astype(int)

    # Hitung Metriks
    recall = recall_score(y_test, y_pred_classified)
    precision = precision_score(y_test, y_pred_classified)
    f1 = f1_score(y_test, y_pred_classified)
    cm = confusion_matrix(y_test, y_pred_classified)

    # Tampilkan Hasil
    print("\n[Metriks Utama Model Judol]")
    print(f"🎯 Recall (Rekal) @ {OPTIMAL_THRESHOLD}: {recall:.4f}")
    print(f"Presisi (Precision) @ {OPTIMAL_THRESHOLD}: {precision:.4f}")
    print(f"F1-Score: {f1:.4f}")
    
    print("\n[Matriks Kebingungan (Confusion Matrix)]")
    print("  | Prediksi Non-Judol | Prediksi Judol")
    print("---|--------------------|----------------")
    print(f"Nyata Non-Judol | {cm[0, 0]:<18} | {cm[0, 1]:<12} (False Positives)")
    print(f"Nyata Judol     | {cm[1, 0]:<18} (False Negatives) | {cm[1, 1]:<12}")
    
    print("\n[Laporan Klasifikasi Terperinci]")
    print(classification_report(y_test, y_pred_classified, target_names=['Non-Judol', 'Judol']))
    
    print("--------------------------------------------------")

if __name__ == '__main__':
    test_judol_model()

# KODE GPT

In [123]:
import pandas as pd
import re
import numpy as np
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential

# =====================================
# 1. LOAD DATA
# =====================================
df = pd.read_csv("production_judol_detection_old.csv")

# pastikan kolom benar
TEXT_COL = "cleaned_comment_text"
LABEL_COL = "target"

df = df[[TEXT_COL, LABEL_COL]].dropna()

# =====================================
# 2. BRAND LIST
# =====================================
brand_list = [
    "cukong", "maxwin", "slot88", "situs gacor", "gacor", "winrate",
    "slot online", "slot dana", "slot deposit", "slot pakai pulsa",
    "gates olympus", "olympus", "pragmatic", "habanero",
]

# bikin regex brand → <BRAND>
brand_regex = re.compile(
    r"\b(" + "|".join([re.escape(b) for b in brand_list]) + r")\b", flags=re.IGNORECASE
)

def replace_brands(text):
    return brand_regex.sub(" <BRAND> ", text)

# =====================================
# 3. DATA AUGMENTATION
# =====================================
def augment_text(text):
    aug = []

    # 1. Swap word random
    words = text.split()
    if len(words) > 3:
        i, j = random.sample(range(len(words)), 2)
        swapped = words.copy()
        swapped[i], swapped[j] = swapped[j], swapped[i]
        aug.append(" ".join(swapped))

    # 2. Insert <BRAND> random
    if random.random() < 0.3:
        idx = random.randint(0, len(words))
        insert_brand = words[:idx] + ["<BRAND>"] + words[idx:]
        aug.append(" ".join(insert_brand))

    # 3. Duplicate random word
    if len(words) > 2:
        k = random.randint(0, len(words)-1)
        dup = words.copy()
        dup.insert(k, words[k])
        aug.append(" ".join(dup))

    return aug


# apply brand replacement
df["text_norm"] = df[TEXT_COL].astype(str).apply(replace_brands)

# generate augment
aug_rows = []
for _, row in df.iterrows():
    text = row["text_norm"]
    label = row[LABEL_COL]

    for augmented in augment_text(text):
        aug_rows.append([augmented, label])

df_aug = pd.DataFrame(aug_rows, columns=["text_norm", LABEL_COL])

# combine original + augmented
df_final = pd.concat([df[["text_norm", LABEL_COL]], df_aug], ignore_index=True)

print("Original:", len(df))
print("Augmented:", len(df_aug))
print("Total:", len(df_final))


Original: 19252
Augmented: 35406
Total: 54658


In [124]:
# =====================================
# TOKENIZER
# =====================================
MAX_WORDS = 20000
MAX_LEN = 100

tokenizer = Tokenizer(num_words=MAX_WORDS, oov_token="<UNK>")
tokenizer.fit_on_texts(df_final["text_norm"])

X = tokenizer.texts_to_sequences(df_final["text_norm"])
X = pad_sequences(X, maxlen=MAX_LEN)

y = df_final[LABEL_COL].values

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train:", len(X_train))
print("Val:", len(X_val))


Train: 43726
Val: 10932


In [125]:
# =====================================
# BUILD MODEL
# =====================================
model = Sequential([
    Embedding(MAX_WORDS, 128, input_length=MAX_LEN),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(32)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

model.summary()

# TRAIN
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=5,
    batch_size=256
)

model.save("judol_detector_lstm.h5")

import pickle
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)


/home/wtf/anaconda3/lib/python3.13/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_24 (Embedding)        │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_20                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_21                │ ?                      │   0 (unbuilt) │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 104s 537ms/step - accuracy: 0.9519 - loss: 0.1546 - val_accuracy: 0.9852 - val_loss: 0.0516
Epoch 2/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 90s 523ms/step - accuracy: 0.9912 - loss: 0.0312 - val_accuracy: 0.9918 - val_loss: 0.0266
Epoch 3/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 90s 528ms/step - accuracy: 0.9946 - loss: 0.0177 - val_accuracy: 0.9929 - val_loss: 0.0202
Epoch 4/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 89s 523ms/step - accuracy: 0.9958 - loss: 0.0122 - val_accuracy: 0.9937 - val_loss: 0.0172
Epoch 5/5
171/171 ━━━━━━━━━━━━━━━━━━━━ 89s 520ms/step - accuracy: 0.9961 - loss: 0.0096 - val_accuracy: 0.9944 - val_loss: 0.0141


In [126]:
import pickle
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

MAX_LEN = 100

# load
model = tf.keras.models.load_model("judol_detector_lstm.h5")
with open("tokenizer.pkl", "rb") as f:
    tokenizer = pickle.load(f)

def normalize(text):
    # brand ke <BRAND>
    text = brand_regex.sub(" <BRAND> ", text)
    return text

def predict(text):
    text = normalize(text)
    seq = tokenizer.texts_to_sequences([text])
    pad = pad_sequences(seq, maxlen=MAX_LEN)
    prob = float(model.predict(pad)[0][0])
    label = 1 if prob >= 0.5 else 0
    return label, prob

# contoh
test = "promo maxwin olympus 100% hari ini"
label, prob = predict(test)
print("Label:", label, "Prob:", prob)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Label: 0 Prob: 0.0020317535381764174
